<a href="https://colab.research.google.com/github/DivsRepo/CIDC/blob/main/Day_7_Ruling_out_Patient_Intake_Form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================
# SECTION 1: UPLOAD TEMPLATE → EXTRACT COORDINATES →
#           GENERATE auto_field_mapping.json + auto_dummy_data.json
# =============================================================
!pip install -q pdfplumber PyPDF2 reportlab

import pdfplumber
from google.colab import files
import json, random, datetime, textwrap
import re

print("📥 Upload your PDF template (Patient Intake form):")
uploaded = files.upload()
template_path = list(uploaded.keys())[0]

# helper - normalize text keys
def norm(s):
    return re.sub(r'[^a-z0-9]', '', s.lower())

# Extract words and rough label groups
coords = []
with pdfplumber.open(template_path) as pdf:
    page = pdf.pages[0]
    words = page.extract_words(use_text_flow=True)  # better grouping when available
    # words contain x0, x1, top, bottom, text
    for w in words:
        coords.append({
            "raw": w["text"],
            "field": w["text"],
            "x": float(w["x0"]),
            "y": float(800 - w["top"]),   # convert to reportlab coords (approx)
            "width": float(w["x1"] - w["x0"]),
            "height": float(w["bottom"] - w["top"]),
            "value": ""
        })

print(f"✅ Extracted {len(coords)} text tokens from page 1")

# Build a quick searchable index by normalized text
index = {i: coords[i] for i in range(len(coords))}

# Define semantic field keywords to search for (adapted to your form)
semantic_keys = {
    "name": ["name", "patientname"],
    "dob": ["dob", "dateofbirth", "birth"],
    "gender": ["gender", "male", "female", "other"],
    "pronouns": ["pronouns"],
    "address": ["address"],
    "city": ["city"],
    "state": ["state"],
    "zip": ["zip", "zipcode", "postal"],
    "preferred_contact": ["preferredcontact", "preferredcontactmethod", "preferred"],
    "phone": ["phone", "phone:"],
    "email": ["email"],
    "textpref": ["text","textmessage","text:"],
    "emergency_name": ["emergency", "emergencycontact", "emergencycontactname"],
    "emergency_phone": ["emergencyphone", "phone:"],
    "relationship": ["relationship", "relationshiptopatient"],
    "insurance_provider": ["provider", "insurance", "insur"],
    "policy_number": ["policy", "policynumber"],
    "group_number": ["groupnumber"],
    "policyholder": ["policyholder"],
    "relation_self": ["self"],
    "reason_for_visit": ["reason", "primaryreason", "visit"],
    "duration": ["howlong", "duration"],
    "treated_before": ["treated", "treatedbefore"],
    "medical_history": ["diabetes","hypertension","hear","heart","asthma","cancer","stroke","medicalhistory"],
    "medications": ["medications", "takingany"],
    "allergies": ["allergies", "allergy"],
    "surgeries": ["surgeries", "hospitalizations", "procedures"],
    "smoke": ["smoke", "tobacco"],
    "alcohol": ["alcohol"],
    "drugs": ["drugs", "recreational"],
    "occupation": ["occupation"],
    "access_issues": ["access", "transportation", "financial"],
    "pharmacy": ["pharmacy"],
    "signature": ["signature"],
    "date_signed": ["date:"]
}

# Function to find nearest token by keyword presence
def find_token_for_keywords(keywords):
    for i, item in index.items():
        nt = norm(item["raw"])
        for kw in keywords:
            if kw in nt:
                return item, i
    return None, None

# Build mapping by searching labels
fields = []
used_indexes = set()
for sem, keys in semantic_keys.items():
    found, idx = find_token_for_keywords(keys)
    if found:
        used_indexes.add(idx)
        # Compute a default x,y for value placement slightly right of label
        x = found["x"] + found["width"] + 6
        y = found["y"]
        fields.append({
            "name": sem,
            "label_text": found["raw"],
            "x": round(x,2),
            "y": round(y,2),
            "width": round(found["width"]*3 + 50,2),
            "height": round(found["height"],2),
            "value": ""
        })

# For remaining obvious checkbox words (like Diabetes, Hypertension etc) create fields if they exist
checkbox_keywords = ["diabetes","hypertension","heart","asthma","cancer","stroke","yes","no","other"]
for i, item in index.items():
    if i in used_indexes: continue
    nt = norm(item["raw"])
    if any(kw in nt for kw in checkbox_keywords):
        fields.append({
            "name": "chk_" + nt[:30],
            "label_text": item["raw"],
            "x": round(item["x"] + 0.5,2),
            "y": round(item["y"],2),
            "width": round(item["width"],2),
            "height": round(item["height"],2),
            "value": ""
        })
        used_indexes.add(i)

# If some important fields missing, add approximate placeholders (safe fallback)
needed = ["name","dob","gender","address","city","state","zip","phone","email","emergency_name","emergency_phone","reason_for_visit","medications","allergies","surgeries","pharmacy","signature","date_signed"]
present_names = {f["name"] for f in fields}
for n in needed:
    if n not in present_names:
        # place fallback at x=120 and staggered y to avoid overlap
        ypos = 700 - 15*len([p for p in fields if p["name"].startswith(n[:4])])
        fields.append({
            "name": n,
            "label_text": n,
            "x": 120.0,
            "y": float(ypos),
            "width": 350.0,
            "height": 12.0,
            "value": ""
        })

mapping_json = {"fields": fields}

# -------------------------
# Generate realistic dummy data (Option A, randomized for checkboxes; Q1=1, Q2=1)
# -------------------------
def random_date(start_year=1940, end_year=2005):
    year = random.randint(start_year, end_year)
    month = random.randint(1,12)
    day = random.randint(1,28)
    return f"{month:02d}/{day:02d}/{year}"

def realistic_dummy_for_field(name):
    name_l = name.lower()
    if "name" == name_l or name_l.endswith("name"):
        return random.choice(["John Doe","Jane Smith","Alex Johnson","Priya Kumar"])
    if name_l == "dob":
        return random_date(1945, 2002)
    if "gender" in name_l:
        return random.choice(["Male","Female","Other"])
    if "pronoun" in name_l:
        return random.choice(["He/Him","She/Her","They/Them"])
    if "address" in name_l:
        return random.choice(["123 Main St","45 Greenway Ave","Unit 7, Sunrise Park"])
    if "city" in name_l:
        return random.choice(["Chennai","Mumbai","Bengaluru","Hyderabad"])
    if "state" in name_l:
        return random.choice(["TN","MH","KA","TG"])
    if "zip" in name_l:
        return random.choice(["600001","400001","560001"])
    if "phone" in name_l:
        return random.choice(["9876543210","9123456789","9998887776"])
    if "email" in name_l:
        return random.choice(["john.doe@example.com","jane.smith@mail.com"])
    if "emergency" in name_l:
        return random.choice(["Mary Doe","Karthik R"])
    if "relationship" in name_l:
        return random.choice(["Spouse","Parent","Sibling","Friend"])
    if "insurance" in name_l or "provider" in name_l:
        return random.choice(["HDFC Ergo","Star Health","Apollo Munich"])
    if "policy" in name_l:
        return random.choice(["POL123456","PN-98765"])
    if "group" in name_l:
        return random.choice(["GRP101","GRP2468"])
    if "reason" in name_l:
        return random.choice(["Lower back pain for 3 weeks","Headache and dizziness","Follow-up for hypertension"])
    if "duration" in name_l:
        return random.choice(["2 weeks","3 days","6 months"])
    if "treated" in name_l:
        return random.choice(["Yes","No"])
    if "medic" in name_l:
        return random.choice(["Metformin 500 mg daily","Paracetamol 500mg prn"])
    if "allerg" in name_l:
        return random.choice(["Penicillin","No known allergies"])
    if "surg" in name_l:
        return random.choice(["Appendectomy (2015)","None"])
    if "smoke" in name_l:
        return random.choice(["No","Yes","Former"])
    if "alcohol" in name_l:
        return random.choice(["Occasionally","No","Yes"])
    if "drugs" in name_l:
        return random.choice(["No","Yes"])
    if "occupation" in name_l:
        return random.choice(["Software Engineer","Teacher","Housewife","Student"])
    if "access" in name_l:
        return random.choice(["No","Yes - transportation issues"])
    if "pharmacy" in name_l:
        return random.choice(["Apollo Pharmacy, Anna Salai"])
    if "signature" in name_l:
        return random.choice(["John Doe"])
    if "date" in name_l:
        return datetime.datetime.now().strftime("%m/%d/%Y")
    # checkboxes or other short labels
    return random.choice(["Yes","No","N/A","Details here"])

# Fill mapping_json with dummy data automatically
dummy = {}
for f in mapping_json["fields"]:
    # for checkboxes (we detect short labels or label_text containing checkbox char)
    label_norm = norm(f["label_text"])
    if any(k in label_norm for k in ["diabetes","hypertension","heart","asthma","cancer","stroke","yesno","yes","no"]):
        # randomized boolean-like
        val = random.choice([True, False])
        f["value"] = val
    else:
        f["value"] = realistic_dummy_for_field(f["name"])

    dummy[f["name"]] = f["value"]

# Save both mapping and dummy data
with open("auto_field_mapping.json", "w") as t:
    json.dump(mapping_json, t, indent=4)

with open("auto_dummy_data.json", "w") as t:
    json.dump(dummy, t, indent=4)

print("✅ Generated files: auto_field_mapping.json and auto_dummy_data.json")
files.download("auto_field_mapping.json")
files.download("auto_dummy_data.json")


📥 Upload your PDF template (Patient Intake form):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (6).pdf
✅ Extracted 255 text tokens from page 1
✅ Generated files: auto_field_mapping.json and auto_dummy_data.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================
# SECTION 2: UPLOAD TEMPLATE + mapping + dummy JSON → HIGHLIGHT coords
# =============================================================
!pip install -q PyPDF2 reportlab

from google.colab import files
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO
import json, textwrap

print("📥 Upload the SAME template PDF (again):")
up_pdf = files.upload()
template_path = list(up_pdf.keys())[0]

print("📥 Upload auto_field_mapping.json (generated in SECTION 1):")
up_map = files.upload()
map_path = list(up_map.keys())[0]

print("📥 Upload auto_dummy_data.json (generated in SECTION 1):")
up_dummy = files.upload()
dummy_path = list(up_dummy.keys())[0]

with open(map_path, "r") as f:
    mapping_json = json.load(f)
with open(dummy_path, "r") as f:
    dummy_json = json.load(f)

# create highlighted overlay
reader = PdfReader(template_path)
packet = BytesIO()
can = Canvas(packet, pagesize=letter)

for item in mapping_json["fields"]:
    x = item["x"]
    y = item["y"]
    w = item.get("width", 120)
    h = item.get("height", 12)
    # draw rectangle and label
    can.setStrokeColorRGB(1, 0, 0)
    can.rect(x-2, y-2, max(w,60), max(h,12), stroke=1, fill=0)
    try:
        can.setFont("Helvetica", 6)
        can.drawString(x, y + 12, str(item["label_text"]))
    except:
        pass

can.showPage()
can.save()
packet.seek(0)

overlay = PdfReader(packet)
writer = PdfWriter()

base = reader.pages[0]
if len(overlay.pages) == 0:
    raise RuntimeError("Overlay was empty — highlight generation failed.")
base.merge_page(overlay.pages[0])
writer.add_page(base)

highlighted_path = "highlighted_coordinates.pdf"
with open(highlighted_path, "wb") as f:
    writer.write(f)

files.download(highlighted_path)
print("🎨 Highlighted coordinates PDF generated and downloaded.")


📥 Upload the SAME template PDF (again):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (7).pdf
📥 Upload auto_field_mapping.json (generated in SECTION 1):


Saving auto_field_mapping.json to auto_field_mapping (3).json
📥 Upload auto_dummy_data.json (generated in SECTION 1):


Saving auto_dummy_data.json to auto_dummy_data (1).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎨 Highlighted coordinates PDF generated and downloaded.


In [ ]:
# =============================================================
# SECTION 3: UPLOAD TEMPLATE + auto_field_mapping.json + auto_dummy_data.json
#           → Auto-fill PDF using dummy data (no manual edits)
# =============================================================
!pip install -q PyPDF2 reportlab

from google.colab import files
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO
import json, textwrap

print("📥 Upload the SAME template PDF (again):")
up_pdf = files.upload()
template_path = list(up_pdf.keys())[0]

print("📥 Upload auto_field_mapping.json:")
up_map = files.upload()
map_path = list(up_map.keys())[0]

print("📥 Upload auto_dummy_data.json:")
up_dummy = files.upload()
dummy_path = list(up_dummy.keys())[0]

with open(map_path, "r") as f:
    mapping_json = json.load(f)
with open(dummy_path, "r") as f:
    dummy_json = json.load(f)

# helper to draw wrapped text inside width
def draw_wrapped(canv, text, x, y, max_width, line_height=12, fontname="Helvetica", fontsize=9):
    lines = textwrap.wrap(str(text), width= int(max_width/6))  # crude width->chars mapping
    canv.setFont(fontname, fontsize)
    for i, line in enumerate(lines):
        canv.drawString(x, y - i*line_height, line)

# Build overlay with values
fill_packet = BytesIO()
can = Canvas(fill_packet, pagesize=letter)

for item in mapping_json["fields"]:
    name = item["name"]
    x = item["x"]
    y = item["y"]
    w = item.get("width", 150)

    # get value from dummy_json; fallback to item["value"]
    val = dummy_json.get(name, item.get("value", ""))
    # If boolean (checkbox-like), render a check at label position
    if isinstance(val, bool):
        if val:
            # draw checkmark slightly left of label (approx checkbox)
            can.setFont("Helvetica", 12)
            can.drawString(x-8, y-2, "✔")
    else:
        # draw wrapped text
        draw_wrapped(can, val, x, y, max_width=w)

# ensure a page exists
can.showPage()
can.save()
fill_packet.seek(0)

overlay = PdfReader(fill_packet)
if len(overlay.pages) == 0:
    raise RuntimeError("Overlay PDF empty — nothing to merge.")

# Merge onto template
template_reader = PdfReader(template_path)
template_page = template_reader.pages[0]
template_page.merge_page(overlay.pages[0])
writer = PdfWriter()
writer.add_page(template_page)

filled_path = "filled_output_dummy.pdf"
with open(filled_path, "wb") as f:
    writer.write(f)

files.download(filled_path)
print("🎉 Filled PDF generated (filled_output_dummy.pdf) and downloaded.")


📥 Upload the SAME template PDF (again):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (8).pdf
📥 Upload auto_field_mapping.json:


Saving auto_field_mapping.json to auto_field_mapping (4).json
📥 Upload auto_dummy_data.json:


Saving auto_dummy_data.json to auto_dummy_data (2).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Filled PDF generated (filled_output_dummy.pdf) and downloaded.


In [ ]:
# =============================================================
# SECTION 1: AUTO-EXTRACTION (LABEL → INPUT LINE ANCHORING)
# =============================================================
!pip install pdfplumber PyPDF2 reportlab

import pdfplumber
from google.colab import files
import json, random, datetime, textwrap
from statistics import mean

print("📥 Upload the Patient Intake PDF:")
uploaded = files.upload()
template_path = list(uploaded.keys())[0]

# ------------------------------------------------------------
# STEP 1: Extract LABELS (using pdfplumber words)
# ------------------------------------------------------------
with pdfplumber.open(template_path) as pdf:
    page = pdf.pages[0]
    words = page.extract_words()

labels = []
for w in words:
    labels.append({
        "text": w["text"],
        "x": float(w["x0"]),
        "y": float(w["top"]),
        "cx": float((w["x0"] + w["x1"]) / 2),
        "cy": float((w["top"] + w["bottom"]) / 2)
    })

# ------------------------------------------------------------
# STEP 2: Extract HORIZONTAL LINES (candidate input fields)
# ------------------------------------------------------------
lines = page.lines + page.edges + page.curves

input_lines = []
for ln in lines:
    x0, y0, x1, y1 = ln["x0"], ln["y0"], ln["x1"], ln["y1"]
    # identify horizontal lines
    if abs(y0 - y1) < 1.5 and abs(x1 - x0) > 30:
        input_lines.append({
            "x0": float(x0),
            "y": float(y0),
            "x1": float(x1),
            "width": float(x1 - x0),
            "cx": float((x0 + x1)/2)
        })

# Remove duplicate/overlapping lines
final_input_lines = []
for ln in input_lines:
    if not any(abs(ln["y"] - o["y"]) < 1 and abs(ln["x0"] - o["x0"]) < 5 for o in final_input_lines):
        final_input_lines.append(ln)

print("Found:", len(labels), "labels")
print("Found:", len(final_input_lines), "input candidate lines")

# ------------------------------------------------------------
# STEP 3: MATCH LABEL → CLOSEST LINE BELOW IT
# ------------------------------------------------------------
fields = []
for lbl in labels:
    lx, ly, lcx = lbl["x"], lbl["y"], lbl["cx"]

    # find candidate lines 10–120 pts below the label
    below = [
        line for line in final_input_lines
        if 0 < (line["y"] - ly) < 120
    ]

    if not below:
        continue

    # choose the closest vertically
    chosen = min(below, key=lambda L: abs(L["y"] - ly))

    fields.append({
        "label": lbl["text"],
        "x": float(chosen["cx"] - 50),
        "y": float(800 - chosen["y"]),   # convert coordinates
        "width": float(chosen["width"]),
        "value": ""
    })

# ------------------------------------------------------------
# STEP 4: GENERATE REALISTIC DUMMY DATA (your Option A)
# ------------------------------------------------------------
def dummy_value(label):
    t = label.lower()
    if "name" in t:
        return random.choice(["John Doe", "Alex Johnson", "Priya Kumar"])
    if "dob" in t:
        return "07/06/1988"
    if "gender" in t:
        return random.choice(["Male", "Female", "Other"])
    if "pronoun" in t:
        return random.choice(["He/Him", "She/Her", "They/Them"])
    if "address" in t:
        return random.choice(["45 Greenway Ave"])
    if "city" in t:
        return random.choice(["Bengaluru", "Chennai"])
    if "state" in t:
        return random.choice(["TN", "KA"])
    if "zip" in t or "zipcode" in t:
        return random.choice(["600001", "400001"])
    if "phone" in t:
        return "9998887776"
    if "email" in t:
        return "john.doe@example.com"
    if "provider" in t:
        return "HDFC Ergo"
    if "policy" in t:
        return "POL123456"
    if "group" in t:
        return "GRP55"
    if "reason" in t:
        return "Headache and dizziness"
    if "long" in t:
        return "3 days"
    if "treated" in t:
        return random.choice(["Yes", "No"])
    if "medication" in t:
        return "Paracetamol 500mg prn"
    if "allerg" in t:
        return "Penicillin"
    if "surg" in t:
        return "Appendectomy (2015)"
    if "occupation" in t:
        return "Software Engineer"
    if "pharmacy" in t:
        return "Apollo Pharmacy, Anna Salai"
    if "date" in t:
        return datetime.datetime.now().strftime("%m/%d/%Y")
    return random.choice(["Yes", "No", "N/A", ""])

dummy_data = {}
for f in fields:
    val = dummy_value(f["label"])
    f["value"] = val
    dummy_data[f["label"]] = val

# Save mapping and dummy data
with open("auto_field_mapping.json","w") as f:
    json.dump({"fields": fields}, f, indent=4)

with open("auto_dummy_data.json","w") as f:
    json.dump(dummy_data, f, indent=4)

print("✅ Field mapping and dummy data generated.")
files.download("auto_field_mapping.json")
files.download("auto_dummy_data.json")


📥 Upload the Patient Intake PDF:


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (9).pdf
Found: 257 labels
Found: 36 input candidate lines
✅ Field mapping and dummy data generated.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =============================================================
# SECTION 2: HIGHLIGHT FIELD COORDINATES
# =============================================================
!pip install PyPDF2 reportlab

from google.colab import files
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO
import json

print("📥 Upload Template PDF:")
u1 = files.upload()
template_path = list(u1.keys())[0]

print("📥 Upload auto_field_mapping.json:")
u2 = files.upload()
mapping_path = list(u2.keys())[0]

with open(mapping_path,"r") as f:
    mapping = json.load(f)["fields"]

packet = BytesIO()
can = Canvas(packet, pagesize=letter)

for item in mapping:
    x = item["x"]
    y = item["y"]
    can.setStrokeColorRGB(1,0,0)
    can.rect(x, y, item["width"], 12, stroke=1, fill=0)
    can.setFont("Helvetica", 7)
    can.drawString(x, y+14, item["label"])

can.save()
packet.seek(0)
overlay = PdfReader(packet)

reader = PdfReader(template_path)
page = reader.pages[0]
page.merge_page(overlay.pages[0])

writer = PdfWriter()
writer.add_page(page)

out_path = "highlighted_fields.pdf"
with open(out_path,"wb") as f:
    writer.write(f)

files.download(out_path)
print("🎨 Highlight PDF ready.")


📥 Upload Template PDF:


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (10).pdf
📥 Upload auto_field_mapping.json:


Saving auto_field_mapping.json to auto_field_mapping (5).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎨 Highlight PDF ready.


In [ ]:
# =============================================================
# Not working
# PATIENT INTAKE FORM — ONE-CELL COLAB
# Auto-detect fields (whitespace), detect checkboxes, semantic classification,
# generate realistic dummy data, highlight, and auto-fill.
# =============================================================

!pip install -q pdfplumber PyPDF2 reportlab rapidfuzz

import pdfplumber
from google.colab import files
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO
import json, random, datetime, textwrap, math
from rapidfuzz import fuzz

print("📥 Upload Patient Intake Form PDF:")
uploaded = files.upload()
template_path = list(uploaded.keys())[0]

# -------------------------
# Helper: normalize text
# -------------------------
def norm(s):
    return "".join([c.lower() for c in s if c.isalnum() or c.isspace()]).strip()

# -------------------------
# STEP 1 — Extract text tokens
# -------------------------
with pdfplumber.open(template_path) as pdf:
    page = pdf.pages[0]
    words = page.extract_words(use_text_flow=True)

labels = []
for w in words:
    labels.append({
        "text": w["text"],
        "x0": float(w["x0"]),
        "x1": float(w["x1"]),
        "y": float(w["top"]),
        "cx": float((w["x0"] + w["x1"]) / 2),
        "cy": float((w["top"] + w["bottom"]) / 2)
    })

# Sort by y (top-to-bottom) then x
labels = sorted(labels, key=lambda x: (x["y"], x["x0"]))

# -------------------------
# STEP 2 — Group labels into horizontal bands (Option A behaviour)
# -------------------------
bands = []
current = []
for lbl in labels:
    if not current:
        current.append(lbl)
        continue
    # same band threshold
    if abs(lbl["y"] - current[-1]["y"]) < 20:
        current.append(lbl)
    else:
        bands.append(current)
        current = [lbl]
if current:
    bands.append(current)

# -------------------------
# STEP 3 — Detect whitespace gaps (candidate input areas) inside each band
# -------------------------
candidates = []
for band in bands:
    band_sorted = sorted(band, key=lambda x: x["x0"])
    for i in range(len(band_sorted)-1):
        left = band_sorted[i]
        right = band_sorted[i+1]
        gap = right["x0"] - left["x1"]
        # threshold tuned for typical intake form layout
        if gap > 60:
            cx = (left["x1"] + right["x0"]) / 2
            cy = (left["y"] + right["y"]) / 2
            candidates.append({
                "band_y": cy,
                "x": float(left["x1"] + 5),
                "y": float(800 - cy),
                "width": float(max(40, gap - 10)),
                "label_left": left["text"],
                "label_right": right["text"],
                "raw_left": left,
                "raw_right": right,
                "type": "text",   # default, may be updated to checkbox
                "value": ""
            })

# -------------------------
# STEP 4 — Detect small checkbox-like shapes
# Approach:
#  - find short horizontal segments and short vertical segments from page.lines/edges,
#  - cluster them into small boxes (center and size),
#  - assign checkbox center → nearest label within 40px
# -------------------------
with pdfplumber.open(template_path) as pdf:
    page = pdf.pages[0]
    # lines may include page.lines, page.edges and page.curves depending on PDF
    raw_lines = []
    if hasattr(page, "lines"):
        raw_lines += page.lines
    if hasattr(page, "edges"):
        raw_lines += page.edges
    if hasattr(page, "curves"):
        raw_lines += page.curves

h_segments = []
v_segments = []
for ln in raw_lines:
    x0,y0,x1,y1 = ln.get("x0", ln.get("x0",0)), ln.get("y0",0), ln.get("x1",0), ln.get("y1",0)
    # horizontal
    if abs(y0 - y1) < 2 and abs(x1-x0) > 0:
        length = abs(x1-x0)
        if 4 < length < 35:   # potential top/bottom of checkbox
            h_segments.append((min(x0,x1), min(y0,y1), max(x0,x1), max(y0,y1)))
    # vertical
    if abs(x0 - x1) < 2 and abs(y1-y0) > 0:
        length = abs(y1-y0)
        if 4 < length < 35:
            v_segments.append((min(x0,x1), min(y0,y1), max(x0,x1), max(y0,y1)))

# find possible box centers by pairing h and v segments close to each other
checkbox_centers = []
for hx0, hy0, hx1, hy1 in h_segments:
    for vx0, vy0, vx1, vy1 in v_segments:
        # crude spatial overlap test: vertical x inside horizontal span, vertical overlaps vertical vicinity
        if vx0 >= hx0 - 2 and vx0 <= hx1 + 2 and abs(vy0 - hy0) < 40:
            # approximate box center
            cx = (hx0 + hx1 + vx0 + vx1)/4
            cy = (hy0 + hy1 + vy0 + vy1)/4
            size = max(hx1-hx0, vy1-vy0)
            checkbox_centers.append({"cx": cx, "cy": cy, "size": size})

# dedupe checkbox centers
dedup = []
for c in checkbox_centers:
    if not any(math.hypot(c["cx"]-d["cx"], c["cy"]-d["cy"]) < 6 for d in dedup):
        dedup.append(c)
checkbox_centers = dedup

# Map checkboxes to nearest label (within vertical/horizontal threshold)
checkbox_fields = []
for cb in checkbox_centers:
    # find nearest label token by euclidean in page coords (use label cx,cy)
    best = None
    bestd = 1e9
    for lbl in labels:
        # convert label y coordinate from pdfplumber (top) to same coordinate system
        dx = cb["cx"] - lbl["cx"]
        dy = cb["cy"] - lbl["y"]
        d = math.hypot(dx, dy)
        if d < bestd:
            bestd = d
            best = lbl
    if best and bestd < 60:
        checkbox_fields.append({
            "label": best["text"],
            "x": float(cb["cx"] - cb["size"]/2),
            "y": float(800 - cb["cy"] - cb["size"]/2),
            "size": float(cb["size"]),
            "type": "checkbox",
            "value": False
        })

# Try to deduplicate checkboxes mapped to same label (if multiple found)
seen_labels = set()
unique_checkboxes = []
for cb in checkbox_fields:
    if cb["label"] not in seen_labels:
        unique_checkboxes.append(cb)
        seen_labels.add(cb["label"])
checkbox_fields = unique_checkboxes

# -------------------------
# STEP 5 — Merge candidate text fields and checkbox fields
# If a checkbox is near a candidate text gap, prefer checkbox
# -------------------------
# mark candidates that overlap checkbox centers and convert them to checkbox if needed
for cb in checkbox_fields:
    # find candidate whose x..x+width includes cb center
    for cand in candidates:
        cb_center_x = cb["x"] + cb["size"]/2
        # convert cand.x (reportlab) to pdf coordinates for comparison: cand.x is already pdf x; cb measured in pdf coords
        if cand["x"] <= cb_center_x <= cand["x"] + cand["width"]:
            # prefer adding separate checkbox field rather than replace; but mark nearby text fields as checkbox_label
            cand["near_checkbox"] = True

# Build final fields list: include text candidates and checkbox_fields
final_fields = []

# convert candidate coords to mapping format used later (reportlab coordinates x,y already computed earlier for text candidates)
for cand in candidates:
    final_fields.append({
        "kind": cand.get("type","text"),
        "subtype": "text",
        "label_left": cand["label_left"],
        "label_right": cand["label_right"],
        "x": cand["x"],
        "y": cand["y"],
        "width": cand["width"],
        "value": cand.get("value","")
    })

# add checkbox fields
for cb in checkbox_fields:
    final_fields.append({
        "kind": "control",
        "subtype": "checkbox",
        "label": cb["label"],
        "x": cb["x"],
        "y": cb["y"],
        "size": cb["size"],
        "value": cb["value"]
    })

# -------------------------
# STEP 6 — Semantic classification (AI-like) using keyword + fuzzy matching
# We prepare a set of semantic categories and keywords, and score each detected field.
# -------------------------
semantic_vocab = {
    "name": ["name", "patient name", "full name"],
    "dob": ["dob", "dateofbirth","birth","birthdate"],
    "gender": ["gender","male","female","other"],
    "pronouns": ["pronouns","he/him","she/her","they/them"],
    "address": ["address","street","addr"],
    "city": ["city","town"],
    "state": ["state","province"],
    "zip": ["zip","postal","postcode"],
    "phone": ["phone","mobile","telephone","cell"],
    "email": ["email","e-mail"],
    "preferred_contact": ["preferred contact","contact method","phone email text"],
    "emergency_contact": ["emergency contact","emergency"],
    "relationship": ["relationship","relation to patient","spouse","parent"],
    "insurance_provider": ["provider","insurance","insurer"],
    "policy_number": ["policy number","policy"],
    "group_number": ["group number","group"],
    "reason_for_visit": ["reason","primary reason","visit"],
    "duration": ["how long","duration","since"],
    "treated_before": ["treated before","have you been treated","yes no"],
    "medical_history": ["diabetes","hypertension","heart disease","asthma","cancer","stroke"],
    "medications": ["medications","taking any medications","list medications"],
    "allergies": ["allergies","allergy","if yes list allergies"],
    "surgeries": ["surgeries","procedures","hospitalizations"],
    "smoking": ["smoke","tobacco","former smoker"],
    "alcohol": ["alcohol","consume alcohol"],
    "drugs": ["recreational drugs"],
    "occupation": ["occupation","job"],
    "pharmacy": ["pharmacy","preferred pharmacy"],
    "signature": ["signature"],
    "date_signed": ["date","signature date","date:"]
}

def classify_label_text(txt):
    txtn = norm(txt)
    best_cat = None
    best_score = -1
    for cat, kws in semantic_vocab.items():
        for kw in kws:
            # fuzzy match
            score = fuzz.partial_ratio(txtn, kw)
            if score > best_score:
                best_score = score
                best_cat = cat
    # if very low score, fallback to 'other'
    if best_score < 30:
        return "other", best_score
    return best_cat, best_score

# classify final_fields
for f in final_fields:
    # choose label text for classification
    if f["subtype"] == "checkbox":
        lab = f.get("label","")
    else:
        lab = (f.get("label_left","") + " " + f.get("label_right","")).strip()
    cat, score = classify_label_text(lab)
    f["semantic"] = cat
    f["score"] = score

# -------------------------
# STEP 7 — ASSIGN DUMMY VALUES BASED ON SEMANTICS (Option A behavior)
# -------------------------
def dummy_by_semantic(sem):
    if sem == "name": return random.choice(["John Doe","Alex Johnson","Priya Kumar"])
    if sem == "dob": return "07/06/1988"
    if sem == "gender": return random.choice(["Male","Female","Other"])
    if sem == "pronouns": return random.choice(["He/Him","She/Her","They/Them"])
    if sem == "address": return "45 Greenway Ave"
    if sem == "city": return "Bengaluru"
    if sem == "state": return "KA"
    if sem == "zip": return "560001"
    if sem == "phone": return "9876543210"
    if sem == "email": return "john.doe@example.com"
    if sem == "preferred_contact": return random.choice(["Phone","Email","Text"])
    if sem == "emergency_contact": return random.choice(["Mary Doe","Karthik R"])
    if sem == "relationship": return random.choice(["Spouse","Parent","Friend"])
    if sem == "insurance_provider": return random.choice(["HDFC Ergo", "Star Health"])
    if sem == "policy_number": return random.choice(["POL123456", "PN-98765"])
    if sem == "group_number": return "GRP101"
    if sem == "reason_for_visit": return random.choice(["Lower back pain for 3 weeks","Headache and dizziness"])
    if sem == "duration": return random.choice(["2 weeks","3 days"])
    if sem == "treated_before": return random.choice(["Yes","No"])
    if sem == "medical_history": return random.choice(["Diabetes","No"])
    if sem == "medications": return random.choice(["Metformin 500 mg daily","Paracetamol 500mg prn"])
    if sem == "allergies": return random.choice(["Penicillin","No known allergies"])
    if sem == "surgeries": return random.choice(["Appendectomy (2015)","None"])
    if sem == "smoking": return random.choice(["No","Former"])
    if sem == "alcohol": return random.choice(["Occasionally","No"])
    if sem == "drugs": return random.choice(["No"])
    if sem == "occupation": return random.choice(["Software Engineer"])
    if sem == "pharmacy": return random.choice(["Apollo Pharmacy, Anna Salai"])
    if sem == "signature": return random.choice(["John Doe"])
    if sem == "date_signed": return datetime.datetime.now().strftime("%m/%d/%Y")
    return ""

# apply dummy values; for checkbox-type fields set boolean
for f in final_fields:
    sem = f.get("semantic","other")
    if f["subtype"] == "checkbox":
        # decide check/uncheck based on semantic (if medical_history etc. randomize)
        if sem in ["medical_history","allergies","medications","smoking","alcohol","drugs"]:
            f["value"] = random.choice([True, False])
        else:
            # generic checkbox: random small chance true
            f["value"] = random.random() < 0.25
    else:
        val = dummy_by_semantic(sem)
        if not val:
            # fallback: if label contains Yes/No likely a treated_before field
            label_text = (f.get("label_left","") + " " + f.get("label_right",""))
            if "yes" in label_text.lower() or "no" in label_text.lower():
                val = random.choice(["Yes","No"])
        f["value"] = val

# -------------------------
# STEP 8 — Save mapping + dummy data files
# -------------------------
mapping_out = {"fields": final_fields}
dummy_out = { (f.get("label") or (f.get("label_left","")+" "+f.get("label_right","")) ).strip(): f["value"] for f in final_fields}

with open("auto_intake_mapping.json","w") as f:
    json.dump(mapping_out, f, indent=4)
with open("auto_intake_dummy.json","w") as f:
    json.dump(dummy_out, f, indent=4)

files.download("auto_intake_mapping.json")
files.download("auto_intake_dummy.json")
print("✅ Mapping + dummy outputs saved & downloaded.")

# -------------------------
# STEP 9 — Highlight detected fields (rectangles for text, boxes for checkboxes)
# -------------------------
reader = PdfReader(template_path)
page0 = reader.pages[0]
packet = BytesIO()
can = Canvas(packet, pagesize=letter)

for f in final_fields:
    if f["subtype"] == "checkbox":
        # draw a small square around detected checkbox
        can.setStrokeColorRGB(0,0.5,0)
        size = f.get("size",12)
        can.rect(f["x"], f["y"], size, size, stroke=1, fill=0)
        can.setFont("Helvetica",6)
        can.drawString(f["x"] + size + 2, f["y"] + size/4, f.get("label",""))
    else:
        can.setStrokeColorRGB(1,0,0)
        can.rect(f["x"], f["y"], f["width"], 10, stroke=1, fill=0)
        can.setFont("Helvetica",6)
        lab = (f.get("label_left","") + " " + f.get("label_right","")).strip()
        can.drawString(f["x"], f["y"] + 12, lab)

can.save()
packet.seek(0)
overlay = PdfReader(packet)
page0.merge_page(overlay.pages[0])

writer = PdfWriter()
writer.add_page(page0)
with open("highlighted_with_checkboxes.pdf","wb") as f:
    writer.write(f)
files.download("highlighted_with_checkboxes.pdf")
print("🎨 Highlighted PDF saved & downloaded.")

# -------------------------
# STEP 10 — Create overlay with filled values (text and checkboxes)
# -------------------------
fill_packet = BytesIO()
can = Canvas(fill_packet, pagesize=letter)

def draw_wrapped(canv, txt, x, y, width, fontsize=9):
    if txt is None:
        return
    chars = max(10, int(width/6))
    lines = textwrap.wrap(str(txt), chars)
    canv.setFont("Helvetica", fontsize)
    for i, line in enumerate(lines):
        canv.drawString(x, y - i*12, line)

for f in final_fields:
    if f["subtype"] == "checkbox":
        if f["value"]:
            # draw a checkmark inside the checkbox
            cx = f["x"] + (f.get("size",10) / 6)
            cy = f["y"] + (f.get("size",10) / 4)
            can.setFont("Helvetica", 12)
            can.drawString(f["x"] + 1, f["y"] + 1, "✓")
    else:
        draw_wrapped(can, f["value"], f["x"], f["y"] + 8, f["width"])

can.save()
fill_packet.seek(0)
overlay_fill = PdfReader(fill_packet)
if len(overlay_fill.pages) == 0:
    raise RuntimeError("Overlay empty — nothing to merge (unexpected)")

reader = PdfReader(template_path)
page_fill = reader.pages[0]
page_fill.merge_page(overlay_fill.pages[0])

writer = PdfWriter()
writer.add_page(page_fill)
with open("filled_intake_with_checkboxes.pdf","wb") as f:
    writer.write(f)

files.download("filled_intake_with_checkboxes.pdf")
print("🎉 Filled PDF (with checkbox marks) generated and downloaded.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.8 MB/s eta 0:00:00
📥 Upload Patient Intake Form PDF:


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (11).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

🎨 Highlighted PDF saved & downloaded.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Filled PDF (with checkbox marks) generated and downloaded.


In [ ]:
# =============================================================
# Not working
# ONE-CELL COLAB: Convert Non-fillable Patient Intake PDF → REAL AcroForm
# With exact-width fields + checkboxes + auto-fill
# FIXED drawImage USING ImageReader (WORKING)
# =============================================================

!apt-get update -qq
!apt-get install -qq -y poppler-utils tesseract-ocr
!pip install -q pdf2image opencv-python-headless reportlab PyPDF2 pillow pdfplumber

import os
from pdf2image import convert_from_path
import cv2, numpy as np
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.units import inch
from PyPDF2 import PdfReader, PdfWriter
from google.colab import files
from io import BytesIO
import pdfplumber
import random, datetime

print("📥 Upload your Patient Intake PDF:")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


# ========== CONFIG ==========
DPI = 300
MIN_WIDTH_PIX = 80
CHECKBOX_MIN = 6
CHECKBOX_MAX = 40
# =============================

# Render page → image
pages = convert_from_path(pdf_path, dpi=DPI, first_page=1, last_page=1)
pil = pages[0].convert("RGB")
img_w, img_h = pil.size

cv = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(cv, cv2.COLOR_BGR2GRAY)
th = cv2.adaptiveThreshold(~gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)

# Horizontal line detection
horiz = th.copy()
hsize = max(1, img_w//40)
kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT,(hsize,1))
horiz=cv2.erode(horiz,kernel_h); horiz=cv2.dilate(horiz,kernel_h)
cnts,_ = cv2.findContours(horiz, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

segments=[]
for c in cnts:
    x,y,w,h=cv2.boundingRect(c)
    if w>MIN_WIDTH_PIX: segments.append([x,y,w,h])
segments=sorted(segments,key=lambda x:(x[1],x[0]))

# Merge touching segments
merged=[]
for s in segments:
    if not merged:
        merged.append(s); continue
    p=merged[-1]
    if abs(s[1]-p[1])<5:
        x0=min(p[0],s[0]); x1=max(p[0]+p[2], s[0]+s[2])
        p[0]=x0; p[2]=x1-x0
    else:
        merged.append(s)

# Checkbox detection
cnts2,_ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
checkboxes=[]
for c in cnts2:
    x,y,w,h=cv2.boundingRect(c)
    if CHECKBOX_MIN<=w<=CHECKBOX_MAX and CHECKBOX_MIN<=h<=CHECKBOX_MAX:
        if 0.7<=w/h<=1.4:
            checkboxes.append([x,y,w,h])

# Dedup checkbox
cb=[]
for c in sorted(checkboxes,key=lambda x:(x[1],x[0])):
    if not cb:
        cb.append(c); continue
    p=cb[-1]
    if abs(c[0]-p[0])<6 and abs(c[1]-p[1])<6: continue
    cb.append(c)

# PDF text extraction
with pdfplumber.open(pdf_path) as pdf:
    page=pdf.pages[0]
    words=page.extract_words()

px_to_pt = 72/DPI
page_w_pt=page.width
page_h_pt=page.height

word_data=[]
for w in words:
    cx=((float(w["x0"])+float(w["x1"]))/2)*px_to_pt*(DPI/72)
    cy=float(w["top"])*px_to_pt*(DPI/72)
    word_data.append({"text":w["text"],"cx":cx,"cy":cy})

# Build fields list
fields=[]
for (x,y,w,h) in merged:
    box_h=25; box_y=y-box_h-2
    if box_y<0: box_y=0
    fields.append({"type":"text","x":x,"y":box_y,"w":w,"h":box_h})

for (x,y,w,h) in cb:
    fields.append({"type":"checkbox","x":x,"y":y,"w":w,"h":h})

# Assign nearest label
for f in fields:
    fx=f["x"]+f["w"]/2
    fy=f["y"]
    cand=[wd for wd in word_data if wd["cy"]<(fy*px_to_pt)]
    if not cand:
        f["label"]=""
    else:
        f["label"]=min(cand,key=lambda wd:abs(wd["cy"]-(fy*px_to_pt))+abs(wd["cx"]-(fx*px_to_pt)))["text"]

# Convert to pt coords
final=[]
for i,f in enumerate(fields):
    x_pt=f["x"]*px_to_pt
    y_pt=(img_h - (f["y"]+f["h"])) * px_to_pt
    w_pt=f["w"]*px_to_pt
    h_pt=f["h"]*px_to_pt
    final.append({
        "name":f"field_{i+1}",
        "type":f["type"],
        "label":f["label"],
        "x":x_pt,"y":y_pt,
        "w":w_pt,"h":h_pt
    })

# Dummy generator
def dummy(label):
    t=(label or "").lower()
    if "name" in t: return random.choice(["John Doe","Alex Johnson"])
    if "dob" in t: return "07/06/1988"
    if "phone" in t: return "9876543210"
    if "email" in t: return "john@example.com"
    if "address" in t: return "45 Greenway Ave"
    if "city" in t: return "Bengaluru"
    if "state" in t: return "KA"
    if "zip" in t: return "560001"
    if "insur" in t: return "HDFC Ergo"
    if "policy" in t: return "POL12345"
    if "reason" in t: return "Headache"
    if "date" in t: return datetime.datetime.now().strftime("%d/%m/%Y")
    return "N/A"

# ================== CREATE REAL ACROFORM ===================
tmpl="rebuilt_fillable_template.pdf"
filled="filled_rebuilt_form.pdf"

# FIXED drawImage: wrap BytesIO in ImageReader
img_bytes=BytesIO()
pil.save(img_bytes,format="PNG")
img_bytes.seek(0)
bg = ImageReader(img_bytes)

c = canvas.Canvas(tmpl, pagesize=(page_w_pt,page_h_pt))
c.drawImage(bg,0,0,width=page_w_pt,height=page_h_pt)

try:
    pdfmetrics.registerFont(TTFont("DejaVu","/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"))
    FONT="DejaVu"
except:
    FONT="Helvetica"

form=c.acroForm

for f in final:
    if f["type"]=="checkbox":
        form.checkbox(name=f["name"],x=f["x"],y=f["y"],size=12,checked=False)
    else:
        form.textfield(name=f["name"], x=f["x"], y=f["y"],
                       width=f["w"], height=max(12,f["h"]),
                       value=dummy(f["label"]),
                       fontName=FONT,fontSize=9,
                       borderWidth=0,forceBorder=False)

c.showPage()
c.save()

# ================== FILL CHECKBOXES ===================
reader = PdfReader(tmpl)
writer = PdfWriter()
writer.append_pages_from_reader(reader)

update={}
for f in final:
    if f["type"]=="checkbox":
        update[f["name"]]="/Yes" if random.random()<0.2 else "/Off"
    else:
        update[f["name"]] = dummy(f["label"])

writer.update_page_form_field_values(writer.pages[0], update)

with open(filled,"wb") as F:
    writer.write(F)

files.download(tmpl)
files.download(filled)

print("🎉 DONE — Template & Filled PDFs Generated")



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
📥 Upload your Patient Intake PDF:


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (17).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 DONE — Template & Filled PDFs Generated


In [ ]:
# =============================================================
# ONE-CELL: DETECT VISUAL FIELDS → CREATE REAL ACROFORM WIDGETS (PyMuPDF)
# Produces: reconstructed_fillable.pdf (editable, invisible borders)
#           reconstructed_filled.pdf  (same, pre-filled with dummy values)
# =============================================================

# Install dependencies
!apt-get update -qq
!apt-get install -qq -y poppler-utils
!pip install -q pdf2image opencv-python-headless pillow pymupdf pdfplumber

import fitz                       # PyMuPDF
from pdf2image import convert_from_path
import cv2, numpy as np
from PIL import Image, ImageFont, ImageDraw
from io import BytesIO
from google.colab import files
import pdfplumber, random, datetime, math, os, sys

print("📥 Upload the form PDF (single PDF page recommended):")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
pdf_path = list(uploaded.keys())[0]

# ---------- Parameters ----------
DPI = 300
MIN_WIDTH_PIX = 80
CHECKBOX_MIN = 6
CHECKBOX_MAX = 40
# -------------------------------

# Render first page to image
pages = convert_from_path(pdf_path, dpi=DPI, first_page=1, last_page=1)
if len(pages) == 0:
    raise RuntimeError("Failed to render PDF to image.")
pil = pages[0].convert("RGB")
img_w, img_h = pil.size
cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
th = cv2.adaptiveThreshold(~gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)

# Detect long horizontal segments (likely underlines / input lines)
horiz = th.copy()
hsize = max(1, img_w//40)
kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT,(hsize,1))
horiz = cv2.erode(horiz,kernel_h)
horiz = cv2.dilate(horiz,kernel_h)
cnts,_ = cv2.findContours(horiz, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

segments=[]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if w > MIN_WIDTH_PIX:
        segments.append((x,y,w,h))
segments = sorted(segments, key=lambda r:(r[1], r[0]))

# Merge close/overlapping segments
merged=[]
for s in segments:
    if not merged:
        merged.append(list(s)); continue
    p = merged[-1]
    if abs(s[1]-p[1]) < 6 and s[0] <= p[0] + p[2] + 8:
        nx0 = min(p[0], s[0])
        nx1 = max(p[0]+p[2], s[0]+s[2])
        p[0] = nx0; p[2] = nx1 - nx0
        p[1] = min(p[1], s[1])
        p[3] = max(p[3], s[3])
    else:
        merged.append(list(s))

# Detect checkbox-like squares
cnts2,_ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
checkboxes=[]
for c in cnts2:
    x,y,w,h = cv2.boundingRect(c)
    if CHECKBOX_MIN <= w <= CHECKBOX_MAX and CHECKBOX_MIN <= h <= CHECKBOX_MAX:
        if 0.7 <= (w/h) <= 1.4:
            checkboxes.append((x,y,w,h))
# dedupe
cb_sorted = sorted(checkboxes, key=lambda r:(r[1], r[0]))
cb_merged=[]
for cb in cb_sorted:
    if not cb_merged:
        cb_merged.append(list(cb)); continue
    p = cb_merged[-1]
    if abs(cb[0]-p[0])<6 and abs(cb[1]-p[1])<6:
        continue
    cb_merged.append(list(cb))

# Extract words from PDF (pdfplumber) to assign labels
with pdfplumber.open(pdf_path) as pdf:
    page0 = pdf.pages[0]
    words = page0.extract_words(use_text_flow=True)

# Convert word coords to pixel space for matching
pt_to_px = DPI/72.0
word_centers=[]
for w in words:
    x0 = float(w['x0'])*pt_to_px
    x1 = float(w['x1'])*pt_to_px
    top = float(w['top'])*pt_to_px
    cx = (x0+x1)/2
    cy = top
    word_centers.append({'text':w['text'],'cx':cx,'cy':cy})

# Build field candidates: merged horizontal segments => text fields; cb_merged => checkboxes
candidates = []
for (x,y,w,h) in merged:
    box_h = max(18, int(h*1.2))
    box_y = max(0, y - box_h - 2)
    candidates.append({'type':'text','x':x,'y':box_y,'w':w,'h':box_h})

for (x,y,w,h) in cb_merged:
    candidates.append({'type':'checkbox','x':x,'y':y,'w':w,'h':h})

# Assign nearest label word above each candidate
for c in candidates:
    fx = c['x'] + c['w']/2
    fy = c['y']
    above = [wd for wd in word_centers if wd['cy'] < fy + 6]
    if not above:
        c['label'] = ""
    else:
        c['label'] = min(above, key=lambda wd: abs(wd['cy'] - fy) + abs(wd['cx'] - fx))['text']

# Convert px -> PDF points (fitz uses points). px_to_pt = 72/DPI
px_to_pt = 72.0 / DPI

# Prepare final field descriptors for fitz
final_fields = []
for i,c in enumerate(candidates):
    x_pt = c['x'] * px_to_pt
    # fitz coordinate: origin at top-left for page.insert_text? But for widget rect we use PDF points bottom-left origin,
    # so compute y from top:
    y_pt = (img_h - (c['y'] + c['h'])) * px_to_pt
    w_pt = c['w'] * px_to_pt
    h_pt = c['h'] * px_to_pt
    final_fields.append({
        'name': f'field_{i+1}',
        'label': c.get('label',''),
        'x': x_pt, 'y': y_pt, 'w': w_pt, 'h': h_pt,
        'type': c['type']
    })

print(f"Detected {len(final_fields)} candidate form widgets (text + checkboxes).")

# ---------- Dummy data generator (semantic heuristics) ----------
def dummy_value(lbl):
    t = (lbl or "").lower()
    if 'name' in t: return random.choice(['John Doe','Priya Kumar','Alex Johnson'])
    if 'dob' in t or 'birth' in t: return '07/06/1988'
    if 'phone' in t or 'cell' in t: return '9876543210'
    if 'email' in t: return 'john@example.com'
    if 'address' in t: return '45 Greenway Ave'
    if 'city' in t: return 'Bengaluru'
    if 'state' in t: return 'KA'
    if 'zip' in t: return '560001'
    if 'policy' in t: return 'POL123456'
    if 'provider' in t or 'insurance' in t: return 'HDFC Ergo'
    if 'reason' in t: return 'Lower back pain'
    if 'med' in t: return 'Metformin 500mg daily'
    if 'allerg' in t: return 'Penicillin'
    if 'date' in t: return datetime.datetime.now().strftime('%m/%d/%Y')
    return 'N/A'

# ---------- Create fillable PDF using PyMuPDF (fitz) ----------
out_template = "reconstructed_fillable.pdf"
out_filled = "reconstructed_filled.pdf"

# We'll open the original PDF and add widgets to page 0
doc = fitz.open(pdf_path)
page = doc[0]

# PyMuPDF widget API: use page.add_widget / doc.newTextWidget etc depending on version.
# We'll attempt to use the documented widget functions; handle API differences via try/except.

widgets_created = []
try:
    # Try using modern 'new form' API (fitz >= 1.19)
    for f in final_fields:
        r = fitz.Rect(f['x'], f['y'], f['x'] + f['w'], f['y'] + f['h'])
        if f['type'] == 'checkbox':
            # create a checkbox widget
            w = page.add_checkbox(name=f['name'], rect=r, value=False)  # returns widget
            widgets_created.append(w)
        else:
            # create a text widget (single-line)
            w = page.add_textfield(name=f['name'], rect=r, text=dummy_value(f['label']), fontname="helv", fontsize=10)
            widgets_created.append(w)
    # Save the template (widgets created but invisible by default)
    doc.save(out_template, incremental=False, encryption=fitz.PDF_ENCRYPT_KEEP)
    doc.close()
    print("Created template with widgets:", out_template)
    # reopen and set field values (some widget APIs allow initial text; if not, we set explicit values)
    doc2 = fitz.open(out_template)
    page2 = doc2[0]
    # Set checkbox/text values using widget objects if accessible
    for f in final_fields:
        if f['type'] == 'checkbox':
            # set randomly checked/un-checked
            val = random.random() < 0.25
            try:
                page2.set_widget_value(f['name'], val)
            except Exception:
                # fallback: find widget and set
                for w in page2.widgets():
                    if w.field_name == f['name']:
                        w.field_value = "Yes" if val else "Off"
        else:
            try:
                page2.set_widget_value(f['name'], dummy_value(f['label']))
            except Exception:
                for w in page2.widgets():
                    if w.field_name == f['name']:
                        w.field_value = dummy_value(f['label'])
    doc2.save(out_filled, incremental=False, encryption=fitz.PDF_ENCRYPT_KEEP)
    doc2.close()
    print("Saved filled widget PDF:", out_filled)
    files.download(out_template)
    files.download(out_filled)
    print("✅ Done — downloaded reconstructed_fillable.pdf and reconstructed_filled.pdf")
except Exception as e:
    # If high-level widget functions are unavailable or fail, fallback to lower-level widget creation
    print("Primary widget API failed (PyMuPDF version mismatch or API difference). Error:", e)
    print("Attempting fallback: create widgets via annotation objects.")
    try:
        doc = fitz.open(pdf_path)
        page = doc[0]
        for f in final_fields:
            r = fitz.Rect(f['x'], f['y'], f['x'] + f['w'], f['y'] + f['h'])
            if f['type'] == 'checkbox':
                # create a widget annotation dictionary manually
                widget = page.add_widget(
                    fitz.PDF_WIDGET_CHECKBOX,  # ordinal - may differ by version
                    r,
                    field_name = f['name'],
                    value = False
                )
            else:
                widget = page.add_widget(
                    fitz.PDF_WIDGET_TEXT,
                    r,
                    field_name = f['name'],
                    value = dummy_value(f['label'])
                )
        doc.save(out_template, incremental=False)
        doc.close()
        print("Fallback widgets saved to", out_template)
        files.download(out_template)
        # try to open and set values for filled output similarly
    except Exception as e2:
        print("Fallback widget creation also failed:", e2)
        print("As last resort I will produce a flattened visual-filled PDF (guaranteed).")
        # Create flattened visual result (draw text onto image and save PDF)
        img = pil.copy()
        draw = ImageDraw.Draw(img)
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 12)
        except:
            font = ImageFont.load_default()
        for f in final_fields:
            txt = dummy_value(f['label'])
            # position in px coords
            x_px = int(f['x'] * DPI / 72.0)
            y_px = int(img_h - (f['y'] * DPI / 72.0) - f['h'] * DPI / 72.0/1.0)
            w_px = int(f['w'] * DPI / 72.0)
            # simple draw
            draw.text((x_px+4, y_px+4), txt, font=font, fill=(0,0,0))
        outbytes = BytesIO()
        img.save(outbytes, format='PNG'); outbytes.seek(0)
        pdfbytes = BytesIO()
        Image.open(outbytes).save(pdfbytes, format='PDF'); pdfbytes.seek(0)
        with open('fallback_flattened_filled.pdf','wb') as F:
            F.write(pdfbytes.read())
        files.download('fallback_flattened_filled.pdf')
        print("Fallback flattened PDF produced and downloaded.")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 98.0 MB/s eta 0:00:00
📥 Upload the form PDF (single PDF page recommended):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (18).pdf
Detected 1098 candidate form widgets (text + checkboxes).
Primary widget API failed (PyMuPDF version mismatch or API difference). Error: 'Page' object has no attribute 'add_textfield'
Attempting fallback: create widgets via annotation objects.
Fallback widget creation also failed: module 'fitz' has no attribute 'PDF_WIDGET_TEXT'
As last resort I will produce a flattened visual-filled PDF (guaranteed).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fallback flattened PDF produced and downloaded.


In [ ]:
# =============================================================
# worked and extracted fields - following code blocks to fill pdf
# ONE-CELL: Pixel-detect fields → create REAL invisible AcroForm widgets (PyMuPDF)
# Finalized tuned version: fields slightly ABOVE underline by 8 px, single-line height
# =============================================================

# Install dependencies (Colab)
!apt-get update -qq
!apt-get install -qq -y poppler-utils
!pip install -q pdf2image opencv-python-headless pillow pymupdf pdfplumber

import fitz                       # PyMuPDF
from pdf2image import convert_from_path
import cv2, numpy as np
from PIL import Image, ImageFont, ImageDraw
from io import BytesIO
from google.colab import files
import pdfplumber, random, datetime, math, sys

print("📥 Upload the Patient Intake PDF (single page recommended):")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
pdf_path = list(uploaded.keys())[0]

# ---------- Parameters ----------
DPI = 300
MIN_WIDTH_PIX = 80
CHECKBOX_MIN = 6
CHECKBOX_MAX = 40
# Visual offset (you chose 8 px above the underline)
Y_OFFSET_PX = 8
# Final visual single-line box height (px)
BOX_HEIGHT_PX = 12
# -------------------------------

# Render first page to high-res image for detection
pages = convert_from_path(pdf_path, dpi=DPI, first_page=1, last_page=1)
if len(pages) == 0:
    raise RuntimeError("Failed to render PDF to image.")
pil = pages[0].convert("RGB")
img_w, img_h = pil.size
cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
th = cv2.adaptiveThreshold(~gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)

# Detect horizontal lines (likely underlines)
horiz = th.copy()
hsize = max(1, img_w // 40)
kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (hsize,1))
horiz = cv2.erode(horiz, kernel_h)
horiz = cv2.dilate(horiz, kernel_h)
cnts,_ = cv2.findContours(horiz, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

segments = []
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if w > MIN_WIDTH_PIX:
        segments.append((x,y,w,h))
segments = sorted(segments, key=lambda r:(r[1], r[0]))

# Merge segments that belong to the same visual line
merged=[]
for s in segments:
    if not merged:
        merged.append(list(s)); continue
    prev = merged[-1]
    if abs(s[1] - prev[1]) < 6 and s[0] <= prev[0] + prev[2] + 8:
        nx0 = min(prev[0], s[0]); nx1 = max(prev[0]+prev[2], s[0]+s[2])
        prev[0] = nx0; prev[2] = nx1 - nx0
        prev[1] = min(prev[1], s[1]); prev[3] = max(prev[3], s[3])
    else:
        merged.append(list(s))

# Detect small checkbox-like squares from contours
cnts2,_ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
checkboxes = []
for c in cnts2:
    x,y,w,h = cv2.boundingRect(c)
    if CHECKBOX_MIN <= w <= CHECKBOX_MAX and CHECKBOX_MIN <= h <= CHECKBOX_MAX:
        ratio = w/h
        if 0.7 <= ratio <= 1.4:
            checkboxes.append((x,y,w,h))
# dedupe
cb_sorted = sorted(checkboxes, key=lambda r:(r[1], r[0]))
cb_merged=[]
for cb in cb_sorted:
    if not cb_merged:
        cb_merged.append(list(cb)); continue
    p = cb_merged[-1]
    if abs(cb[0]-p[0])<6 and abs(cb[1]-p[1])<6:
        continue
    cb_merged.append(list(cb))

# Extract PDF words (pdfplumber) to find labels
with pdfplumber.open(pdf_path) as pdf:
    page0 = pdf.pages[0]
    words = page0.extract_words(use_text_flow=True)

# Convert words to pixel centers for matching
pt_to_px = DPI / 72.0
word_centers = []
for w in words:
    x0 = float(w['x0']) * pt_to_px
    x1 = float(w['x1']) * pt_to_px
    top = float(w['top']) * pt_to_px
    cx = (x0 + x1) / 2.0
    cy = top
    word_centers.append({'text': w['text'], 'cx': cx, 'cy': cy})

# Build candidate boxes, apply Y_OFFSET_PX and BOX_HEIGHT_PX
candidates = []
for (x,y,w,h) in merged:
    box_h = BOX_HEIGHT_PX
    box_y = max(0, y - box_h - Y_OFFSET_PX)   # move ABOVE the underline by chosen offset
    candidates.append({'type':'text','x':int(x),'y':int(box_y),'w':int(w),'h':int(box_h)})

for (x,y,w,h) in cb_merged:
    candidates.append({'type':'checkbox','x':int(x),'y':int(y),'w':int(w),'h':int(h)})

# Assign nearest label above each candidate
for c in candidates:
    fx = c['x'] + c['w']/2
    fy = c['y']
    above = [wd for wd in word_centers if wd['cy'] < fy + 6]
    if not above:
        c['label'] = ""
    else:
        # score by vertical closeness then horizontal
        c['label'] = min(above, key=lambda wd: abs(wd['cy'] - fy) + abs(wd['cx'] - fx))['text']

# Convert pixel coords -> PDF points (fitz uses points, origin bottom-left)
px_to_pt = 72.0 / DPI
final_fields = []
for i,c in enumerate(candidates):
    x_pt = c['x'] * px_to_pt
    # compute bottom-left y in points: convert top-left px coordinate to PDF bottom-left origin
    y_pt = (img_h - (c['y'] + c['h'])) * px_to_pt
    w_pt = c['w'] * px_to_pt
    h_pt = c['h'] * px_to_pt
    final_fields.append({
        'name': f'field_{i+1}',
        'label': c.get('label',''),
        'x': x_pt, 'y': y_pt, 'w': w_pt, 'h': h_pt,
        'type': c['type']
    })

print(f"Detected {len(final_fields)} candidate widgets (text + checkboxes).")

# ---------- Dummy data heuristics ----------
def dummy_value(lbl):
    t = (lbl or "").lower()
    if 'name' in t: return random.choice(['John Doe','Priya Kumar','Alex Johnson'])
    if 'dob' in t or 'birth' in t: return '07/06/1988'
    if 'phone' in t or 'cell' in t: return '9876543210'
    if 'email' in t: return 'john@example.com'
    if 'address' in t: return '45 Greenway Ave'
    if 'city' in t: return 'Bengaluru'
    if 'state' in t: return 'KA'
    if 'zip' in t or 'postal' in t: return '560001'
    if 'policy' in t: return 'POL123456'
    if 'provider' in t or 'insurance' in t: return 'HDFC Ergo'
    if 'reason' in t: return 'Lower back pain for 3 weeks'
    if 'med' in t: return 'Metformin 500mg daily'
    if 'allerg' in t: return 'Penicillin'
    if 'date' in t: return datetime.datetime.now().strftime('%m/%d/%Y')
    return 'N/A'

# ---------- Create widgets with PyMuPDF ----------
out_template = "reconstructed_fillable.pdf"
out_filled = "reconstructed_filled.pdf"

doc = fitz.open(pdf_path)
page = doc[0]

created = []
try:
    # Modern API attempt: page.add_textfield, page.add_checkbox (most recent pymupdf)
    for f in final_fields:
        r = fitz.Rect(f['x'], f['y'], f['x'] + f['w'], f['y'] + f['h'])
        if f['type'] == 'checkbox':
            # create checkbox (unchecked by default)
            try:
                page.add_checkbox(name=f['name'], rect=r, value=False)   # fitz >= 1.21 style
            except Exception:
                # fallback to generic widget creation later
                page.add_widget(fitz.PDF_WIDGET_CHECKBOX, r, field_name=f['name'])
        else:
            # create single-line text widget with small font; invisible border by setting border width=0
            try:
                page.add_textfield(name=f['name'], rect=r, value=dummy_value(f['label']), fontsize=9, fontname="helv")
            except Exception:
                # fallback to generic widget creation
                page.add_widget(fitz.PDF_WIDGET_TEXT, r, field_name=f['name'], value=dummy_value(f['label']))
        created.append(f['name'])

    # Save template (widgets created)
    doc.save(out_template, incremental=False)
    doc.close()

    # Re-open and set values (robust method)
    doc2 = fitz.open(out_template)
    page2 = doc2[0]
    # set widget values using widget iteration (works across versions)
    for w in page2.widgets():
        fname = getattr(w, "field_name", None) or getattr(w, "name", None)
        if not fname:
            continue
        # find our field descriptor
        fd = next((x for x in final_fields if x['name'] == fname), None)
        if not fd:
            continue
        if fd['type'] == 'checkbox':
            val = random.random() < 0.25
            try:
                w.field_value = "Yes" if val else "Off"
            except Exception:
                try:
                    w.set_value("Yes" if val else "Off")
                except:
                    pass
        else:
            try:
                w.field_value = dummy_value(fd['label'])
            except Exception:
                try:
                    w.set_value(dummy_value(fd['label']))
                except:
                    pass

    doc2.save(out_filled, incremental=False)
    doc2.close()

    files.download(out_template)
    files.download(out_filled)
    print("✅ Done — reconstructed_fillable.pdf and reconstructed_filled.pdf generated and downloaded.")

except Exception as ex_primary:
    # Widget API differences fallback branch
    print("Primary widget API failed / partial. Error:", ex_primary)
    print("Attempting fallback widget creation using annotations (best-effort).")
    try:
        doc = fitz.open(pdf_path)
        page = doc[0]
        for f in final_fields:
            r = fitz.Rect(f['x'], f['y'], f['x'] + f['w'], f['y'] + f['h'])
            if f['type'] == 'checkbox':
                try:
                    page.add_widget(fitz.PDF_WIDGET_CHECKBOX, r, field_name=f['name'])
                except Exception:
                    pass
            else:
                try:
                    page.add_widget(fitz.PDF_WIDGET_TEXT, r, field_name=f['name'], value=dummy_value(f['label']))
                except Exception:
                    pass
        doc.save(out_template, incremental=False)
        files.download(out_template)
        print("Fallback template saved (may require viewer-specific rendering).")
    except Exception as ex_fallback:
        print("Fallback also failed:", ex_fallback)
        # Last resort: produce a flattened visual filled PDF so you still get usable output
        print("As last resort creating a flattened visual filled PDF.")
        img = pil.copy()
        draw = ImageDraw.Draw(img)
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 12)
        except:
            font = ImageFont.load_default()
        for f in final_fields:
            txt = dummy_value(f['label'])
            x_px = int(f['x'] * DPI / 72.0)
            y_px = int(img_h - (f['y'] * DPI / 72.0) - f['h'] * DPI / 72.0)
            draw.text((x_px+4, y_px+2), txt, font=font, fill=(0,0,0))
        outbytes = BytesIO()
        img.save(outbytes, format='PNG'); outbytes.seek(0)
        pdfbytes = BytesIO()
        Image.open(outbytes).save(pdfbytes, format='PDF'); pdfbytes.seek(0)
        with open('fallback_flattened_filled.pdf','wb') as F:
            F.write(pdfbytes.read())
        files.download('fallback_flattened_filled.pdf')
        print("Fallback flattened PDF produced and downloaded.")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
📥 Upload the Patient Intake PDF (single page recommended):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (19).pdf
Detected 1098 candidate widgets (text + checkboxes).
Primary widget API failed / partial. Error: module 'fitz' has no attribute 'PDF_WIDGET_TEXT'
Attempting fallback widget creation using annotations (best-effort).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fallback template saved (may require viewer-specific rendering).


In [ ]:
# ============================================================
# Not working
# AUTO-FILL EXISTING ACROFORM FIELDS (PyMuPDF)
# ============================================================

!pip install -q pymupdf

import fitz  # PyMuPDF
from google.colab import files

print("Upload your reconstructed_fillable.pdf:")
uploaded = files.upload()
form_path = list(uploaded.keys())[0]

# ------------------------------------------------------------
# 1. OPEN THE FORM
# ------------------------------------------------------------
doc = fitz.open(form_path)
page = doc[0]

# ------------------------------------------------------------
# 2. DEFINE THE DATA YOU WANT TO FILL
#    (This dictionary keys MUST match field names in your PDF)
# ------------------------------------------------------------
data = {
    "field_1": "John Doe",
    "field_2": "07/06/1988",
    "field_3": "Male",
    "field_4": "He/Him",
    "field_5": "45 Greenway Ave",
    "field_6": "Bengaluru",
    "field_7": "KA",
    "field_8": "560001",
    "field_9": "9876543210",
    "field_10": "john@example.com",
    "field_11": "Alex Johnson",
    "field_12": "Spouse",
    "field_13": "HDFC Ergo",
    "field_14": "POL123456",
    "field_15": "GRP101",
    "field_16": "Priya Kumar",
    "field_17": "Back Pain",
    "field_18": "3 weeks",
    "field_19": "Yes",
    "field_20": "Diabetes",
    "field_21": "Metformin 500mg",
    "field_22": "Penicillin",
    "field_23": "Appendectomy (2015)"
}

# ------------------------------------------------------------
# 3. SET EACH FIELD VALUE
# ------------------------------------------------------------
for widget in page.widgets():
    fname = widget.field_name
    if fname in data:
        try:
            widget.field_value = data[fname]
        except:
            widget.set_value(data[fname])

# ------------------------------------------------------------
# 4. SAVE THE OUTPUT
# ------------------------------------------------------------
output_path = "autofilled_output.pdf"
doc.save(output_path)
doc.close()

files.download(output_path)
print("🎉 Autofilled PDF generated:", output_path)


Upload your reconstructed_fillable.pdf:


Saving reconstructed_fillable.pdf to reconstructed_fillable (1).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Autofilled PDF generated: autofilled_output.pdf


In [ ]:
import fitz
from google.colab import files

print("Upload the reconstructed_fillable.pdf:")
uploaded = files.upload()
form_path = list(uploaded.keys())[0]

doc = fitz.open(form_path)
page = doc[0]

print("\n=== FIELD NAMES DETECTED ===\n")
for w in page.widgets():
    print(w.field_name)


Upload the reconstructed_fillable.pdf:


Saving reconstructed_fillable.pdf to reconstructed_fillable (2).pdf

=== FIELD NAMES DETECTED ===

Name
DOB MMDDYYYY
Gender
Other
Other
Other
undefined_2
undefined_3
undefined_4
undefined_5
undefined_6
Address
City
State
Zip
Phone
Email
Preferred Contact Method Phone Email Text
undefined_7
undefined_8
undefined_9
Emergency Contact Name
Phone_2
Relationship to Patient
Provider
Policy number
Group Number
Policyholder Name
undefined_10
undefined_11
undefined_12
undefined_13
undefined_14
Primary Reason for Visit
How long have you had this issue
undefined_15
undefined_16
Do you have any of the following conditions Check all that apply
undefined_17
Diabetes
Hypertension
Heart Disease
Asthma
Cancer
Stroke
undefined_18
undefined_19
If yes list medications
Do you have any allergies
Yes
If yes list allergies
hospitalizations
Yes_2
If yes list procedures and dates
undefined_20
undefined_21
undefined_22
undefined_23
undefined_24
undefined_25
undefined_26
undefined_27
Occupation
undefined_28
undefi

In [ ]:
data = {
    # Patient identification
    "Name": "John Doe",
    "DOB MMDDYYYY": "07/06/1988",
    "Gender": "Male",
    "Other": "",

    # Contact info
    "Address": "45 Greenway Ave",
    "City": "Bengaluru",
    "State": "KA",
    "Zip": "560001",
    "Phone": "9876543210",
    "Email": "john@example.com",
    "Preferred Contact Method Phone Email Text": "Phone",

    # Emergency Contact
    "Emergency Contact Name": "Alex Johnson",
    "Phone_2": "9876500012",
    "Relationship to Patient": "Spouse",

    # Insurance
    "Provider": "HDFC Ergo",
    "Policy number": "POL123456",
    "Group Number": "GRP101",
    "Policyholder Name": "Priya Kumar",

    # Primary visit
    "Primary Reason for Visit": "Lower back pain",
    "How long have you had this issue": "3 weeks",

    # Conditions
    "Do you have any of the following conditions Check all that apply": "",
    "Diabetes": "Yes",
    "Hypertension": "No",
    "Heart Disease": "No",
    "Asthma": "No",
    "Cancer": "No",
    "Stroke": "No",

    # Medications & allergies
    "If yes list medications": "Metformin 500mg daily",
    "Do you have any allergies": "Yes",
    "If yes list allergies": "Penicillin",
    "hospitalizations": "Appendectomy (2015)",
    "If yes list procedures and dates": "Appendectomy - 2015",

    # Additional
    "Occupation": "Software Engineer",
    "If yes please describe": "",
    "Phone Number": "9876543210",
    "Address_2": "Bengaluru, KA",
    "I confirm that the information provided is accurate to the best of my knowledge": "Yes",
    "Date": "12/02/2025",
    "Signature39": "John Doe"
}


In [ ]:
import fitz  # PyMuPDF
from google.colab import files

print("Upload reconstructed_fillable.pdf:")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Open the PDF
doc = fitz.open(pdf_path)
page = doc[0]

# Autofill the fields
for w in page.widgets():
    fname = w.field_name
    if fname in data:
        try:
            w.field_value = data[fname]
        except:
            w.set_value(data[fname])

# Save output
outname = "autofilled_patient_intake.pdf"
doc.save(outname)
doc.close()

files.download(outname)
print("✅ Autofilled PDF generated successfully!")


Upload reconstructed_fillable.pdf:


Saving reconstructed_fillable.pdf to reconstructed_fillable (3).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Autofilled PDF generated successfully!


In [ ]:
# Not working
# === ONE-CELL: robust widget-fill + ensure NeedAppearances + flattened fallback ===
!pip install -q pymupdf PyPDF2 pdfplumber

import fitz, io, os
from google.colab import files
from PyPDF2 import PdfReader, PdfWriter
from PIL import Image, ImageDraw, ImageFont
import pdfplumber, datetime, random

print("Upload your reconstructed_fillable.pdf (the template you built):")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
in_pdf = list(uploaded.keys())[0]

# 1) Inspect fields (print them) for debugging
doc = fitz.open(in_pdf)
print("\nWIDGET LIST (name -> type):")
for w in doc[0].widgets():
    t = getattr(w, "field_type", None) or getattr(w, "ft", None)
    print(w.field_name, " | type:", t)
doc.close()

# 2) Provide a mapping for values: adapt these to fill exactly the fields you saw earlier.
#    If your field names differ, update the keys. I prepared a best-effort map from your field list.
data = {
    "Name": "John Doe",
    "DOB MMDDYYYY": "07/06/1988",
    "Gender": "Male",
    "Address": "45 Greenway Ave",
    "City": "Bengaluru",
    "State": "KA",
    "Zip": "560001",
    "Phone": "9876543210",
    "Email": "john@example.com",
    "Preferred Contact Method Phone Email Text": "Phone",
    "Emergency Contact Name": "Alex Johnson",
    "Phone_2": "9876500012",
    "Relationship to Patient": "Spouse",
    "Provider": "HDFC Ergo",
    "Policy number": "POL123456",
    "Group Number": "GRP101",
    "Policyholder Name": "Priya Kumar",
    "Primary Reason for Visit": "Lower back pain",
    "How long have you had this issue": "3 weeks",
    "Diabetes": "Yes",
    "Hypertension": "No",
    "Heart Disease": "No",
    "Asthma": "No",
    "Cancer": "No",
    "Stroke": "No",
    "If yes list medications": "Metformin 500mg daily",
    "Do you have any allergies": "Yes",
    "If yes list allergies": "Penicillin",
    "hospitalizations": "Appendectomy (2015)",
    "Occupation": "Software Engineer",
    "Phone Number": "9876543210",
    "Address_2": "Bengaluru, KA",
    "I confirm that the information provided is accurate to the best of my knowledge": "Yes",
    "Date": datetime.datetime.now().strftime("%m/%d/%Y"),
    "Signature39": "John Doe"
}

# 3) Set widget values using PyMuPDF (fitz)
doc = fitz.open(in_pdf)
page = doc[0]
set_count = 0
for w in page.widgets():
    name = w.field_name
    # prefer exact match; fallback: try simplified name keys
    if name in data:
        val = data[name]
    else:
        # try fuzzy fallback: lower-case compare up to short keys
        nm = name.lower() if name else ""
        val = None
        for k in data:
            if k.lower() in nm or nm in k.lower():
                val = data[k]; break
    if val is not None:
        try:
            # try attribute set
            w.field_value = str(val)
        except Exception:
            try:
                w.set_value(str(val))
            except Exception:
                pass
        set_count += 1

# Save intermediate file (widget values set)
filled_widgets_path = "filled_with_widgets_unsaved.pdf"
doc.save(filled_widgets_path, incremental=False)
doc.close()
print(f"\nSet values on {set_count} widgets (attempted). Saved as {filled_widgets_path}")

# 4) Ensure /AcroForm/NeedAppearances = True using PyPDF2 (so viewers regenerate appearances)
from PyPDF2.generic import BooleanObject, DictionaryObject, NameObject, IndirectObject

reader = PdfReader(filled_widgets_path)
writer = PdfWriter()
writer.append_pages_from_reader(reader)
root = writer._root_object

# Define NameObjects for keys
acro_form_name = NameObject("/AcroForm")
need_appearances_name = NameObject("/NeedAppearances")
true_boolean = BooleanObject(True)

# Find or create the AcroForm dictionary
acro_form_dict = None

# Check if AcroForm exists in the root
if acro_form_name in root:
    acro_form_dict = root[acro_form_name]
    # If it's an IndirectObject, resolve it to the actual dictionary
    if isinstance(acro_form_dict, IndirectObject):
        acro_form_dict = reader.get_object(acro_form_dict)
elif acro_form_name in reader.trailer["/Root"]: # Fallback: check original PDF's trailer
    acro_form_dict = reader.trailer["/Root"][acro_form_name]
    if isinstance(acro_form_dict, IndirectObject):
        acro_form_dict = reader.get_object(acro_form_dict)

if acro_form_dict is None:
    # Create a new AcroForm dictionary if it was not found
    acro_form_dict = DictionaryObject()
    # Add it to the root and make it an indirect object managed by the writer
    root[acro_form_name] = writer._add_object(acro_form_dict)
elif not isinstance(acro_form_dict, DictionaryObject):
    # If it's found but not a DictionaryObject (e.g., malformed PDF), create a new one
    acro_form_dict = DictionaryObject()
    root[acro_form_name] = writer._add_object(acro_form_dict)

# Now ensure /NeedAppearances is set to True
acro_form_dict.update({need_appearances_name: true_boolean})

out_widgets_pdf = "filled_with_widgets.pdf"
with open(out_widgets_pdf, "wb") as f:
    writer.write(f)

print("Wrote", out_widgets_pdf, "with /NeedAppearances=True (viewer should render values).")

# 5) As a guaranteed fallback, produce a flattened visual PDF (stamped text on image) to ensure you get a filled PDF now
#    This ensures you never end the day without usable output.
print("Producing flattened fallback (visual) PDF as guaranteed output...")
# Render page to PIL again and stamp text at each widget bbox (use widget positions from filled_widgets_path)
flat_doc = fitz.open(filled_widgets_path)
flat_page = flat_doc[0]
pix = flat_page.get_pixmap(dpi=150)
img = Image.open(io.BytesIO(pix.tobytes("png")))
draw = ImageDraw.Draw(img)

# small font
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 11)
except:
    font = ImageFont.load_default()

# stamp using widget rects from the original fill attempt (use page.widgets())
for w in flat_page.widgets():
    nm = w.field_name or ""
    val = None
    if nm in data:
        val = str(data[nm])
    else:
        # fallback fuzzy same as earlier
        nm_l = nm.lower()
        for k in data:
            if k.lower() in nm_l or nm_l in k.lower():
                val = str(data[k]); break
    if not val:
        continue
    # widget rect is in PDF points (bottom-left origin)
    r = w.rect
    # convert to image px (pix.width, pix.height correspond to dpi used)
    # fitz pixmap used 150 DPI above; conversion: pt * (DPI_pix / 72)
    DPI_PIX = 150.0
    scale = DPI_PIX / 72.0
    x = int(r.x0 * scale)
    y = int((flat_page.rect.height - r.y1) * scale)  # invert y
    draw.text((x + 2, y + 2), val, font=font, fill=(0,0,0))

# save flattened pdf
buf = io.BytesIO()
img.save(buf, format="PDF")
buf.seek(0)
with open("flattened_fallback.pdf","wb") as f:
    f.write(buf.read())

print("Saved flattened fallback as flattened_fallback.pdf")

# 6) Offer downloads
files.download(out_widgets_pdf)
files.download("flattened_fallback.pdf")
print("✅ Downloads started: filled_with_widgets.pdf and flattened_fallback.pdf")

Upload your reconstructed_fillable.pdf (the template you built):


Saving reconstructed_fillable.pdf to reconstructed_fillable (5).pdf

WIDGET LIST (name -> type):
Name  | type: 7
DOB MMDDYYYY  | type: 7
Gender  | type: 7
Other  | type: 5
Other  | type: 5
Other  | type: 5
undefined_2  | type: 2
undefined_3  | type: 2
undefined_4  | type: 2
undefined_5  | type: 2
undefined_6  | type: 7
Address  | type: 7
City  | type: 7
State  | type: 7
Zip  | type: 7
Phone  | type: 7
Email  | type: 7
Preferred Contact Method Phone Email Text  | type: 7
undefined_7  | type: 2
undefined_8  | type: 2
undefined_9  | type: 2
Emergency Contact Name  | type: 7
Phone_2  | type: 7
Relationship to Patient  | type: 7
Provider  | type: 7
Policy number  | type: 7
Group Number  | type: 7
Policyholder Name  | type: 7
undefined_10  | type: 2
undefined_11  | type: 2
undefined_12  | type: 2
undefined_13  | type: 2
undefined_14  | type: 7
Primary Reason for Visit  | type: 7
How long have you had this issue  | type: 7
undefined_15  | type: 2
undefined_16  | type: 2
Do you have any of the

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Downloads started: filled_with_widgets.pdf and flattened_fallback.pdf


In [ ]:
# =============================================================
# ONE-CELL: Set widget values (invisible widgets) + force NeedAppearances
# and produce guaranteed flattened PDF with text placed exactly in widget rects.
# Paste and run this entire block in Colab.
# =============================================================

!pip install -q pymupdf PyPDF2 pdfplumber pillow

import fitz, io, os, math
from google.colab import files
from PyPDF2 import PdfReader, PdfWriter
from PIL import Image, ImageDraw, ImageFont
import pdfplumber, datetime, random, sys

print("Upload your reconstructed_fillable.pdf (the template you built earlier):")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
template_path = list(uploaded.keys())[0]

# ---------- Prepare fill data (edit values here if you want different text) ----------
data = {
    "Name": "John Doe",
    "DOB MMDDYYYY": "07/06/1988",
    "Gender": "Male",
    "Address": "45 Greenway Ave",
    "City": "Bengaluru",
    "State": "KA",
    "Zip": "560001",
    "Phone": "9876543210",
    "Email": "john@example.com",
    "Preferred Contact Method Phone Email Text": "Phone",
    "Emergency Contact Name": "Alex Johnson",
    "Phone_2": "9876500012",
    "Relationship to Patient": "Spouse",
    "Provider": "HDFC Ergo",
    "Policy number": "POL123456",
    "Group Number": "GRP101",
    "Policyholder Name": "Priya Kumar",
    "Primary Reason for Visit": "Lower back pain",
    "How long have you had this issue": "3 weeks",
    "Diabetes": "Yes",
    "Hypertension": "No",
    "Heart Disease": "No",
    "Asthma": "No",
    "Cancer": "No",
    "Stroke": "No",
    "If yes list medications": "Metformin 500mg daily",
    "Do you have any allergies": "Yes",
    "If yes list allergies": "Penicillin",
    "hospitalizations": "Appendectomy (2015)",
    "If yes list procedures and dates": "Appendectomy - 2015",
    "Occupation": "Software Engineer",
    "Phone Number": "9876543210",
    "Address_2": "Bengaluru, KA",
    "I confirm that the information provided is accurate to the best of my knowledge": "Yes",
    "Date": datetime.datetime.now().strftime("%m/%d/%Y"),
    "Signature39": "John Doe"
}

# ---------- Step A: open PDF, enumerate widgets ----------
doc = fitz.open(template_path)
page = doc[0]
widgets = list(page.widgets())
print(f"\nFound {len(widgets)} widgets on page 0.\n")
for i,w in enumerate(widgets, start=1):
    nm = getattr(w, "field_name", None) or getattr(w, "name", None)
    ft = getattr(w, "field_type", None) or getattr(w, "ft", None)
    rect = w.rect if hasattr(w, "rect") else None
    print(f"{i:02d}. name='{nm}' type='{ft}' rect={rect}")

# ---------- Step B: Attempt to set widget values (PyMuPDF) ----------
set_count = 0
for w in widgets:
    name = getattr(w, "field_name", None) or getattr(w, "name", None)
    if not name:
        continue
    # exact match first
    val = None
    if name in data:
        val = data[name]
    else:
        # fuzzy fallback: match by containment (lowercase)
        nl = name.lower()
        for k in data:
            if k.lower() in nl or nl in k.lower():
                val = data[k]; break
    if val is None:
        # leave blank
        continue
    try:
        # try direct attribute set
        w.field_value = str(val)
        set_count += 1
    except Exception:
        try:
            w.set_value(str(val))
            set_count += 1
        except Exception:
            # last attempt: write into widget annotation's dictionary (best-effort)
            try:
                # raw update via annotation object (low-level)
                ann = w._annot
                if ann:
                    ann.set_pdf_object({"/V": str(val)})
                    set_count += 1
            except Exception:
                pass

# Save interim PDF with widget values written (may not render in viewer without NeedAppearances)
interim_pdf = "widgets_filled_unsaved.pdf"
doc.save(interim_pdf, incremental=False)
doc.close()
print(f"\nAttempted to set values on {set_count} widgets. Interim file saved: {interim_pdf}")

# ---------- Step C: force NeedAppearances = True (PyPDF2) ----------
try:
    reader = PdfReader(interim_pdf)
    writer = PdfWriter()
    writer.append_pages_from_reader(reader)
    # set NeedAppearances in AcroForm
    root = writer._root_object
    # If document already has AcroForm, update; otherwise create it
    try:
        if "/AcroForm" in root:
            ac = root["/AcroForm"]
            # set NeedAppearances true
            if isinstance(ac, dict):
                ac.update({"/NeedAppearances": True})
            else:
                root["/AcroForm"].update({"/NeedAppearances": True})
        else:
            # create a minimal AcroForm object
            na = writer._add_object({"/NeedAppearances": True})
            root.update({"/AcroForm": na})
    except Exception:
        # fallback: set at root
        root.update({"/NeedAppearances": True})
    out_widgets_pdf = "widgets_filled_with_appearances.pdf"
    with open(out_widgets_pdf, "wb") as f:
        writer.write(f)
    print(f"Saved {out_widgets_pdf} with /NeedAppearances = true.")
except Exception as e:
    print("Failed to set NeedAppearances:", e)
    out_widgets_pdf = interim_pdf

# ---------- Step D: produce guaranteed flattened / exact-filled PDF (pixel perfect) ----------
# We'll render the interim PDF (with widget bbox positions) to an image and stamp the same values exactly onto image
flat_doc = fitz.open(interim_pdf)
flat_page = flat_doc[0]

# Render at high DPI to preserve clarity
DPI_PIX = 300
pix = flat_page.get_pixmap(dpi=DPI_PIX)
img = Image.open(io.BytesIO(pix.tobytes("png")))
draw = ImageDraw.Draw(img)

# font: choose a small legible font
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 11)
except:
    font = ImageFont.load_default()

# For each widget get its rectangle and stamp the value at the same coords
count_flat = 0
for w in flat_page.widgets():
    name = getattr(w, "field_name", None) or getattr(w, "name", None)
    if not name:
        continue
    # pick value same logic as above
    val = None
    if name in data:
        val = data[name]
    else:
        nl = name.lower()
        for k in data:
            if k.lower() in nl or nl in k.lower():
                val = data[k]; break
    if not val:
        continue
    # w.rect is in PDF points (72pt = 1 inch). Convert to pixel coordinates used by pix
    # pix.width corresponds to page width at DPI_PIX => pt_to_px = DPI_PIX/72
    pt_to_px = DPI_PIX / 72.0
    r = w.rect
    x_px = int(r.x0 * pt_to_px)
    # note: fitz rect y coords: origin bottom-left; pix image origin top-left; convert:
    y_px = int((flat_page.rect.height - r.y1) * pt_to_px)
    # stamp text inside box with small margin
    tx = x_px + 3
    ty = y_px + 1
    # wrap text to fit box width if needed
    box_w_px = int((r.x1 - r.x0) * pt_to_px) - 6
    # simple wrapping
    words = str(val).split()
    lines = []
    line = ""
    for token in words:
        test = (line + " " + token).strip()
        tw = draw.textsize(test, font=font)[0]
        if tw <= box_w_px:
            line = test
        else:
            if line:
                lines.append(line)
            line = token
    if line:
        lines.append(line)
    # draw up to 3 lines
    for i,ln in enumerate(lines[:3]):
        draw.text((tx, ty + i*12), ln, font=font, fill=(0,0,0))
    count_flat += 1

# Save flattened pdf
buf = io.BytesIO()
img.save(buf, format="PDF")
buf.seek(0)
with open("flattened_exact_filled.pdf","wb") as f:
    f.write(buf.read())

print(f"\nFlattened exact filled PDF created with {count_flat} stamped values: flattened_exact_filled.pdf")

# ---------- Step E: provide downloads ----------
files.download(out_widgets_pdf)
files.download("flattened_exact_filled.pdf")

print("\nDONE. Two files downloaded (or starting):")
print(" 1) editable attempt:", out_widgets_pdf)
print(" 2) guaranteed visual:", "flattened_exact_filled.pdf")
print("\nIf the 'editable' file still shows blanks in your viewer, open the flattened_exact_filled.pdf — it is guaranteed pixel-perfect.")
print("\nAlso paste the console output here (the widget list and number of set widgets) if you still see empties and I'll patch the final low-level widget construction.")


Upload your reconstructed_fillable.pdf (the template you built earlier):


Saving reconstructed_fillable.pdf to reconstructed_fillable (6).pdf

Found 71 widgets on page 0.

01. name='Name' type='7' rect=Rect(42.84000015258789, 48.5999755859375, 427.79998779296875, 65.280029296875)
02. name='DOB MMDDYYYY' type='7' rect=Rect(527.4000244140625, 48.5999755859375, 602.52001953125, 65.280029296875)
03. name='Gender' type='7' rect=Rect(16.68000030517578, 80.15997314453125, 83.4000015258789, 92.52001953125)
04. name='Other' type='5' rect=Rect(55.560001373291016, 70.79998779296875, 63.84000015258789, 78.239990234375)
05. name='Other' type='5' rect=Rect(91.08000183105469, 70.79998779296875, 99.36000061035156, 78.239990234375)
06. name='Other' type='5' rect=Rect(138.24000549316406, 70.79998779296875, 146.52000427246094, 78.239990234375)
07. name='undefined_2' type='2' rect=Rect(280.44000244140625, 70.79998779296875, 288.7200012207031, 78.239990234375)
08. name='undefined_3' type='2' rect=Rect(327.6000061035156, 70.79998779296875, 335.8800048828125, 78.239990234375)
09. 

AttributeError: 'ImageDraw' object has no attribute 'textsize'

In [ ]:
# =============================================================
# ONE-CELL: Detect placeholders → Fill exact positions → Flattened guaranteed output
# Also attempts to set real widgets (best-effort). Copy & run in Colab.
# =============================================================

!apt-get update -qq
!apt-get install -qq -y poppler-utils
!pip install -q pdf2image opencv-python-headless pillow pymupdf pdfplumber PyPDF2

from pdf2image import convert_from_path
import cv2, numpy as np, math, io, random, datetime
from PIL import Image, ImageDraw, ImageFont
import pdfplumber
from google.colab import files
import fitz          # PyMuPDF
from PyPDF2 import PdfReader, PdfWriter

print("Upload the Patient Intake template PDF (the blank form):")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
pdf_path = list(uploaded.keys())[0]

# -------- Config (tune if needed) ----------
DPI = 300
MIN_LINE_WIDTH_PX = 80        # min horizontal segment width to treat as an input underline
BOX_HEIGHT_PX = 12            # height of the "text area" to stamp text into
Y_OFFSET_ABOVE_LINE = 8       # px above underline to place text (you previously chose 8)
FONT_SIZE = 11
# -------------------------------------------

# Render page1 -> image
pages = convert_from_path(pdf_path, dpi=DPI, first_page=1, last_page=1)
pil = pages[0].convert("RGB")
img_w, img_h = pil.size
cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
th = cv2.adaptiveThreshold(~gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)

# Detect long horizontal segments (likely underlines)
h = th.copy()
hsize = max(1, img_w//40)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (hsize,1))
h = cv2.erode(h, kernel); h = cv2.dilate(h, kernel)
cnts, _ = cv2.findContours(h, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

segments = []
for c in cnts:
    x,y,w,hh = cv2.boundingRect(c)
    if w > MIN_LINE_WIDTH_PX:
        segments.append((x,y,w,hh))
segments = sorted(segments, key=lambda r:(r[1], r[0]))

# Merge nearby/fragmented segments into single underlines
merged = []
for s in segments:
    if not merged:
        merged.append(list(s)); continue
    p = merged[-1]
    if abs(s[1]-p[1]) < 6 and s[0] <= p[0]+p[2]+8:
        nx0 = min(p[0], s[0]); nx1 = max(p[0]+p[2], s[0]+s[2])
        p[0] = nx0; p[2] = nx1 - nx0
        p[1] = min(p[1], s[1]); p[3] = max(p[3], s[3])
    else:
        merged.append(list(s))

# Extract words via pdfplumber for label mapping
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]
    words = page.extract_words(use_text_flow=True)
# convert word centers to px
pt_to_px = DPI / 72.0
word_centers = []
for w in words:
    x0 = float(w["x0"]) * pt_to_px
    x1 = float(w["x1"]) * pt_to_px
    top = float(w["top"]) * pt_to_px
    cx = (x0 + x1)/2.0
    cy = top
    word_centers.append({"text": w["text"], "cx": cx, "cy": cy})

# Build candidate boxes (text fields) slightly above underline
candidates = []
for (x,y,w,hh) in merged:
    box_h = BOX_HEIGHT_PX
    box_y = max(0, y - box_h - Y_OFFSET_ABOVE_LINE)
    candidates.append({"type":"text","x":int(x),"y":int(box_y),"w":int(w),"h":int(box_h)})

# Also detect checkbox-like squares (small boxes)
cnts2, _ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
checkboxes = []
for c in cnts2:
    x,y,w,h = cv2.boundingRect(c)
    if 6 <= w <= 40 and 6 <= h <= 40 and 0.7 <= (w/h) <= 1.4:
        checkboxes.append((x,y,w,h))
# dedupe
cb_sorted = sorted(checkboxes, key=lambda r:(r[1], r[0]))
cb_merged = []
for cb in cb_sorted:
    if not cb_merged:
        cb_merged.append(list(cb)); continue
    p = cb_merged[-1]
    if abs(cb[0]-p[0])<6 and abs(cb[1]-p[1])<6:
        continue
    cb_merged.append(list(cb))
for (x,y,w,h) in cb_merged:
    candidates.append({"type":"checkbox","x":int(x),"y":int(y),"w":int(w),"h":int(h)})

# Assign nearest label above each candidate
for c in candidates:
    fx = c["x"] + c["w"]/2.0
    fy = c["y"]
    above = [wd for wd in word_centers if wd["cy"] < fy + 6]
    if not above:
        c["label"] = ""
    else:
        c["label"] = min(above, key=lambda wd: abs(wd["cy"] - fy) + abs(wd["cx"] - fx))["text"]

# Prepare data mapping (edit these values if you want other content)
data = {
    "Name": "John Doe",
    "DOB": "07/06/1988",
    "Gender": "Male",
    "Address": "45 Greenway Ave",
    "City": "Bengaluru",
    "State": "KA",
    "Zip": "560001",
    "Phone": "9876543210",
    "Email": "john@example.com",
    "Preferred": "Phone",
    "Emergency": "Alex Johnson",
    "Relation": "Spouse",
    "Provider": "HDFC Ergo",
    "Policy": "POL123456",
    "Group": "GRP101",
    "Policyholder": "Priya Kumar",
    "Primary Reason for Visit": "Lower back pain",
    "How long have you had this issue": "3 weeks",
    "Diabetes": "Yes",
    "If yes list medications": "Metformin 500mg",
    "Do you have any allergies": "Yes",
    "If yes list allergies": "Penicillin",
    "hospitalizations": "Appendectomy (2015)",
    "Occupation": "Software Engineer",
    "Date": datetime.datetime.now().strftime("%m/%d/%Y"),
    "Signature": "John Doe"
}

# Attempt to set widget values using PyMuPDF (best-effort)
try:
    doc = fitz.open(pdf_path)
    page_f = doc[0]
    set_count = 0
    for w in page_f.widgets():
        name = getattr(w, "field_name", None) or getattr(w, "name", None) or ""
        if not name:
            continue
        # try exact match or fuzzy match
        val = None
        for k in data:
            if k.lower() in name.lower() or name.lower() in k.lower():
                val = data[k]; break
        if val is None:
            continue
        try:
            w.field_value = str(val); set_count += 1
        except:
            try:
                w.set_value(str(val)); set_count += 1
            except:
                pass
    interim = "widgets_try_filled.pdf"
    doc.save(interim, incremental=False)
    doc.close()
except Exception as e:
    interim = None
    print("PyMuPDF widget attempt failed:", e)

# Now produce the guaranteed flattened exact output (stamp text into exact rectangles)
# Render interim (if widget file exists) else render original
render_path = interim if interim else pdf_path
doc2 = fitz.open(render_path)
pg = doc2[0]
# render at high DPI for crispness
DPI_PIX = 300
pix = pg.get_pixmap(dpi=DPI_PIX)
img = Image.open(io.BytesIO(pix.tobytes("png")))
draw = ImageDraw.Draw(img)

# font selection and fallback
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", FONT_SIZE)
except:
    font = ImageFont.load_default()

# stamp each candidate value into its box
pt_to_px = DPI_PIX / 72.0
count = 0
for c in candidates:
    # choose value by label mapping heuristics
    lbl = (c.get("label") or "").lower()
    val = None
    # try matching by common keywords
    for key in data:
        if key.lower() in lbl or lbl in key.lower():
            val = data[key]; break
    # fallback heuristics: some labels may be identical to pdf words like "Phone" or "Email"
    if val is None:
        if "name" in lbl: val = data.get("Name")
        elif "dob" in lbl or "date" in lbl: val = data.get("Date")
        elif "city" in lbl: val = data.get("City")
        elif "state" in lbl: val = data.get("State")
        elif "zip" in lbl: val = data.get("Zip")
        elif "phone" in lbl: val = data.get("Phone")
        elif "email" in lbl: val = data.get("Email")
        else:
            # if checkbox, maybe put 'Yes' for some common ones
            if c["type"] == "checkbox":
                val = ""  # leave checkbox blank for flattened; user likely toggles in editor
            else:
                val = None
    if not val:
        continue
    # compute pixel coords
    x_px = int(c["x"])
    y_px = int(c["y"])
    # our coordinates are from the page image (top-left origin), so stamp directly
    # wrap text into box width
    box_w = c["w"] - 6
    words = str(val).split()
    lines = []
    line = ""
    for token in words:
        test = (line + " " + token).strip()
        # use textbbox to measure
        bbox = draw.textbbox((0,0), test, font=font)
        tw = bbox[2] - bbox[0]
        if tw <= box_w:
            line = test
        else:
            if line:
                lines.append(line)
            line = token
    if line:
        lines.append(line)
    # draw lines (max 3)
    for i,ln in enumerate(lines[:3]):
        draw.text((x_px + 4, y_px + 1 + i*12), ln, font=font, fill=(0,0,0))
    count += 1

# save flattened exact PDF
out_flat = "flattened_exact_filled.pdf"
buf = io.BytesIO()
img.save(buf, format="PDF")
buf.seek(0)
with open(out_flat, "wb") as f:
    f.write(buf.read())

# Offer downloads (widgets_try_filled.pdf may be None if widget step failed)
if interim:
    files.download(interim)
files.download(out_flat)
print(f"Done — stamped {count} placeholders. Downloads starting:")
if interim:
    print(f" editable attempt (may or may not be editable in your viewer): {interim}")
print(f" flattened guaranteed output: {out_flat}")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Upload the Patient Intake template PDF (the blank form):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (20).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done — stamped 394 placeholders. Downloads starting:
 editable attempt (may or may not be editable in your viewer): widgets_try_filled.pdf
 flattened guaranteed output: flattened_exact_filled.pdf


In [ ]:
# =============================================================
# Scattered filled data
# ONE-CELL: Produce a guaranteed flattened (non-editable) filled PDF
# For Patient_Intake_form-Fillable.pdf — pixel-perfect stamping into blanks
# =============================================================

!apt-get update -qq
!apt-get install -qq -y poppler-utils
!pip install -q pdf2image opencv-python-headless pillow pdfplumber

from pdf2image import convert_from_path
import cv2, numpy as np, io, math, random, datetime
from PIL import Image, ImageDraw, ImageFont
import pdfplumber
from google.colab import files

print("Upload the Patient Intake template PDF (the blank form):")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
pdf_path = list(uploaded.keys())[0]

# ---------- PARAMETERS (tweakable) ----------
DPI = 300                     # rendering DPI
MIN_LINE_WIDTH_PX = 70        # detect horizontal segments wider than this
MERGE_VERTICAL_TOL = 8        # merge nearby segments vertically
BOX_HEIGHT_PX = 14            # height used to stamp text (px)
Y_OFFSET_ABOVE_LINE = 8       # px above detected underline to place text
FONT_SIZE = 11
MAX_LINES_PER_BOX = 3
# ---------------------------------------------

print("Rendering PDF page to image (high-res)...")
pages = convert_from_path(pdf_path, dpi=DPI, first_page=1, last_page=1)
if len(pages) == 0:
    raise SystemExit("Failed to render PDF.")
pil = pages[0].convert("RGB")
img_w, img_h = pil.size
cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)

# Enhance lines: adaptive threshold invert to get lines as blobs
th = cv2.adaptiveThreshold(~gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)

# Detect horizontal lines using morphology
h = th.copy()
hsize = max(1, img_w//40)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (hsize,1))
h = cv2.erode(h, kernel)
h = cv2.dilate(h, kernel)
cnts, _ = cv2.findContours(h, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

segments = []
for c in cnts:
    x,y,w,hh = cv2.boundingRect(c)
    if w > MIN_LINE_WIDTH_PX:
        segments.append((x,y,w,hh))
segments = sorted(segments, key=lambda r:(r[1], r[0]))

# Merge fragmented segments on same visual line
merged = []
for s in segments:
    if not merged:
        merged.append(list(s)); continue
    p = merged[-1]
    # same horizontal band (close y) and overlapping/adjacent
    if abs(s[1] - p[1]) < MERGE_VERTICAL_TOL and s[0] <= p[0] + p[2] + 12:
        nx0 = min(p[0], s[0])
        nx1 = max(p[0]+p[2], s[0]+s[2])
        p[0] = nx0; p[2] = nx1 - nx0
        p[1] = min(p[1], s[1]); p[3] = max(p[3], s[3])
    else:
        merged.append(list(s))

print(f"Detected {len(merged)} long horizontal segments (candidate fields).")

# Also detect checkbox-like small squares
cnts2, _ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
checkboxes = []
for c in cnts2:
    x,y,w,h = cv2.boundingRect(c)
    if 6 <= w <= 40 and 6 <= h <= 40:
        aspect = w/h
        if 0.7 <= aspect <= 1.4:
            checkboxes.append((x,y,w,h))
# dedupe checkbox results
cb_sorted = sorted(checkboxes, key=lambda r:(r[1], r[0]))
cb_merged = []
for cb in cb_sorted:
    if not cb_merged:
        cb_merged.append(list(cb)); continue
    p = cb_merged[-1]
    if abs(cb[0]-p[0]) < 6 and abs(cb[1]-p[1]) < 6:
        continue
    cb_merged.append(list(cb))

print(f"Detected {len(cb_merged)} checkbox-like boxes.")

# Use pdfplumber to extract words (to map label → field)
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]
    words = page.extract_words(use_text_flow=True)

pt_to_px = DPI / 72.0
word_centers = []
for w in words:
    x0 = float(w['x0']) * pt_to_px
    x1 = float(w['x1']) * pt_to_px
    top = float(w['top']) * pt_to_px
    cx = (x0 + x1) / 2.0
    cy = top
    word_centers.append({'text': w['text'], 'cx': cx, 'cy': cy, 'x0': x0, 'x1': x1})

# Build "candidate boxes" — for each merged underline create a box slightly above it
candidates = []
for (x,y,w,hh) in merged:
    box_h = BOX_HEIGHT_PX
    box_y = max(0, y - box_h - Y_OFFSET_ABOVE_LINE)
    candidates.append({'type':'text','x':int(x),'y':int(box_y),'w':int(w),'h':int(box_h)})

# Add checkboxes as candidates
for (x,y,w,h) in cb_merged:
    candidates.append({'type':'checkbox','x':int(x),'y':int(y),'w':int(w),'h':int(h)})

# Attach nearest label above each candidate (if any)
for c in candidates:
    fx = c['x'] + c['w']/2.0
    fy = c['y']
    above = [wd for wd in word_centers if wd['cy'] < fy + 6]
    if not above:
        c['label'] = ""
    else:
        c['label'] = min(above, key=lambda wd: abs(wd['cy'] - fy) + abs(wd['cx'] - fx))['text']

# Heuristic data map — change values here if you want different content
data = {
    "name": "John Doe",
    "dob": "07/06/1988",
    "gender": "Male",
    "address": "45 Greenway Ave",
    "city": "Bengaluru",
    "state": "KA",
    "zip": "560001",
    "phone": "9876543210",
    "email": "john@example.com",
    "preferred": "Phone",
    "emergency": "Alex Johnson",
    "relationship": "Spouse",
    "provider": "HDFC Ergo",
    "policy": "POL123456",
    "group": "GRP101",
    "policyholder": "Priya Kumar",
    "reason": "Lower back pain",
    "duration": "3 weeks",
    "diabetes": "Yes",
    "medications": "Metformin 500mg daily",
    "allergies": "Penicillin",
    "hospitalizations": "Appendectomy (2015)",
    "occupation": "Software Engineer",
    "date": datetime.datetime.now().strftime("%m/%d/%Y"),
    "signature": "John Doe"
}

# Create an image copy and draw text into candidate boxes
DPI_OUT = 300
pix_scale = 1.0
img_out = pil.copy()
draw = ImageDraw.Draw(img_out)

# choose font
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", FONT_SIZE)
except:
    font = ImageFont.load_default()

# Helper: measure text width using textbbox
def text_width(text, font):
    bbox = draw.textbbox((0,0), text, font=font)
    return bbox[2] - bbox[0]

# Stamp values into boxes using label matching heuristics
stamped = 0
for c in candidates:
    lbl = (c.get('label') or "").lower()
    value = None

    # Semantic matching
    for k in data:
        if k in lbl or lbl in k:
            value = data[k]; break

    # Some direct keywords
    if not value:
        if "name" in lbl: value = data.get("name")
        elif "dob" in lbl or "date" in lbl: value = data.get("dob") or data.get("date")
        elif "city" in lbl: value = data.get("city")
        elif "state" in lbl: value = data.get("state")
        elif "zip" in lbl or "postal" in lbl: value = data.get("zip")
        elif "phone" in lbl or "cell" in lbl: value = data.get("phone")
        elif "email" in lbl: value = data.get("email")
        elif "provider" in lbl or "insurance" in lbl: value = data.get("provider")
        elif "policy" in lbl: value = data.get("policy")
        elif "group" in lbl: value = data.get("group")
        elif "policyholder" in lbl: value = data.get("policyholder")
        elif "reason" in lbl: value = data.get("reason")
        elif "how long" in lbl or "how long have" in lbl: value = data.get("duration")
        elif "med" in lbl: value = data.get("medications")
        elif "allerg" in lbl: value = data.get("allergies")
        elif "hospital" in lbl: value = data.get("hospitalizations")
        elif "occupation" in lbl: value = data.get("occupation")
        elif "signature" in lbl or "sign" in lbl: value = data.get("signature")
        else:
            value = None

    # For checkboxes, we leave empty (user can edit manually) — can draw a small check if desired
    if c['type'] == 'checkbox':
        # optionally place a small tick for matched labels that are affirmative
        if value and str(value).lower() in ("yes","y","true"):
            # draw a small check mark centered
            cx = c['x'] + c['w']//2
            cy = c['y'] + c['h']//2
            draw.line((cx-4, cy, cx, cy+6), fill=(0,0,0), width=2)
            draw.line((cx, cy+6, cx+8, cy-6), fill=(0,0,0), width=2)
            stamped += 1
        continue

    if not value:
        continue

    # Prepare wrapped lines to fit box width
    box_w = max(20, c['w'] - 8)
    words = str(value).split()
    lines = []
    line = ""
    for token in words:
        test = (line + " " + token).strip()
        tw = text_width(test, font)
        if tw <= box_w:
            line = test
        else:
            if line:
                lines.append(line)
            line = token
    if line:
        lines.append(line)
    if not lines:
        continue

    # Draw up to MAX_LINES_PER_BOX lines
    x_px = c['x'] + 4
    y_px = c['y'] + 2
    line_h = int(FONT_SIZE * 1.15)
    for i,ln in enumerate(lines[:MAX_LINES_PER_BOX]):
        draw.text((x_px, y_px + i*line_h), ln, fill=(0,0,0), font=font)
    stamped += 1

print(f"Stamped {stamped} candidate placeholders with values.")

# Save flattened PDF (single-page)
out_path = "flattened_exact_filled.pdf"
buf = io.BytesIO()
img_out.save(buf, format="PDF")
buf.seek(0)
with open(out_path, "wb") as f:
    f.write(buf.read())

files.download(out_path)
print("Done — downloaded flattened_exact_filled.pdf (guaranteed visual output).")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Upload the Patient Intake template PDF (the blank form):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (21).pdf
Rendering PDF page to image (high-res)...
Detected 39 long horizontal segments (candidate fields).
Detected 1059 checkbox-like boxes.
Stamped 15 candidate placeholders with values.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done — downloaded flattened_exact_filled.pdf (guaranteed visual output).


In [ ]:
# =============================================================
# ONE-CELL: Robust flattened filler (guaranteed visual output)
# Uses multiple detection heuristics (shaded areas, table rectangles, underlines)
# Run in Colab, upload Patient_Intake_form-Fillable.pdf when prompted
# Output: flattened_exact_filled.pdf
# =============================================================

!apt-get update -qq
!apt-get install -qq -y poppler-utils
!pip install -q pdf2image opencv-python-headless pillow pdfplumber

from pdf2image import convert_from_path
import cv2, numpy as np, io, math, random, datetime
from PIL import Image, ImageDraw, ImageFont
import pdfplumber
from google.colab import files

print("Upload the Patient Intake template PDF (the blank form):")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
pdf_path = list(uploaded.keys())[0]

# ---------- PARAMETERS (tweak later if needed) ----------
DPI = 300
DETECT_MIN_W = 50          # min width px for a candidate box
BOX_HEIGHT_PX = 14
Y_OFFSET_ABOVE_LINE = 6
FONT_SIZE = 11
MAX_LINES_PER_BOX = 3
MERGE_DIST = 12
# -------------------------------------------------------

# Render first page at high DPI
pages = convert_from_path(pdf_path, dpi=DPI, first_page=1, last_page=1)
if len(pages) == 0:
    raise SystemExit("Failed to render PDF.")
pil = pages[0].convert("RGB")
img_w, img_h = pil.size
cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)

# 1) Heuristic A: detect pale/shaded fill regions (light-blue background fields)
# Convert to HSV and find low-saturation/high-value regions (likely form fill backgrounds)
hsv = cv2.cvtColor(cv_img, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)
# threshold: low saturation and high value -> light-ish regions
_, mask_v = cv2.threshold(v, 190, 255, cv2.THRESH_BINARY)
_, mask_s = cv2.threshold(s, 60, 255, cv2.THRESH_BINARY_INV)
mask_light = cv2.bitwise_and(mask_v, mask_s)
# morphological cleanup
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
mask_light = cv2.morphologyEx(mask_light, cv2.MORPH_CLOSE, kernel, iterations=2)
cnts, _ = cv2.findContours(mask_light, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
candidatesA = []
for c in cnts:
    x,y,w,hc = cv2.boundingRect(c)
    if w > DETECT_MIN_W and hc > 8:
        candidatesA.append((x,y,w,hc))

# 2) Heuristic B: detect large rectangular cells (table layout)
# Edge + contour -> find rectangles
edges = cv2.Canny(gray, 50, 150)
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
edges_closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel2, iterations=1)
cnts2, _ = cv2.findContours(edges_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
candidatesB = []
for c in cnts2:
    x,y,w,hc = cv2.boundingRect(c)
    area = w*hc
    if w > DETECT_MIN_W and area > 2000 and hc > 8:
        candidatesB.append((x,y,w,hc))

# 3) Heuristic C: underline detection (long horizontal segments)
th = cv2.adaptiveThreshold(~gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)
hsize = max(1, img_w//50)
kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (hsize,1))
horiz = cv2.erode(th, kernel_h); horiz = cv2.dilate(horiz, kernel_h)
cnts3, _ = cv2.findContours(horiz, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
candidatesC = []
for c in cnts3:
    x,y,w,hc = cv2.boundingRect(c)
    if w > DETECT_MIN_W:
        # make small box above line
        box_h = BOX_HEIGHT_PX
        box_y = max(0, y - box_h - Y_OFFSET_ABOVE_LINE)
        candidatesC.append((x, box_y, w, box_h))

# Combine candidates and dedupe by overlap/centroid proximity
all_cands = candidatesA + candidatesB + candidatesC
# normalize ints and sort
all_cands = [(int(x),int(y),int(w),int(h)) for (x,y,w,h) in all_cands]
all_cands = sorted(all_cands, key=lambda r:(r[1], r[0]))

# merge overlapping/nearby boxes
merged = []
for (x,y,w,hc) in all_cands:
    if w < DETECT_MIN_W:
        continue
    if not merged:
        merged.append([x,y,w,hc]); continue
    px,py,pw,ph = merged[-1]
    if (abs(x - px) < MERGE_DIST and abs(y - py) < MERGE_DIST) or (x <= px + pw + MERGE_DIST and abs(y - py) < MERGE_DIST):
        nx0 = min(px, x)
        ny0 = min(py, y)
        nx1 = max(px+pw, x+w)
        ny1 = max(py+ph, y+hc)
        merged[-1] = [nx0, ny0, nx1-nx0, ny1-ny0]
    else:
        merged.append([x,y,w,hc])

# Filter tiny or full-width table frames that are not fields (optional)
filtered = []
for (x,y,w,hc) in merged:
    # ignore if almost full width header/footer lines
    if w > img_w*0.95 and hc < 6:
        continue
    filtered.append({"type":"text","x":x,"y":y,"w":w,"h":hc})

# Add checkbox candidates (small squares)
cnts_chk, _ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
check_boxes = []
for c in cnts_chk:
    x,y,w,hc = cv2.boundingRect(c)
    if 6 <= w <= 50 and 6 <= hc <= 50 and 0.7 <= (w/hc) <= 1.4:
        check_boxes.append((x,y,w,hc))
# dedupe checkboxes and add
cb_sorted = sorted(check_boxes, key=lambda r:(r[1], r[0]))
cb_final = []
for cb in cb_sorted:
    if not cb_final:
        cb_final.append(cb); continue
    px,py,pw,ph = cb_final[-1]
    if abs(cb[0]-px) < 6 and abs(cb[1]-py) < 6:
        continue
    cb_final.append(cb)
for (x,y,w,hc) in cb_final:
    filtered.append({"type":"checkbox","x":int(x),"y":int(y),"w":int(w),"h":int(hc)})

print(f"Candidates detected: {len(filtered)} (merged heuristics)")

# Extract words from PDF to map label → box
with pdfplumber.open(pdf_path) as pdf:
    p = pdf.pages[0]
    words = p.extract_words(use_text_flow=True)

pt_to_px = DPI/72.0
word_centers = []
for w in words:
    x0 = float(w["x0"])*pt_to_px; x1 = float(w["x1"])*pt_to_px
    top = float(w["top"])*pt_to_px
    cx = (x0+x1)/2.0
    word_centers.append({"text":w["text"], "cx":cx, "cy":top, "x0":x0, "x1":x1})

# Assign nearest label to each candidate (prefer above)
for c in filtered:
    cx = c["x"] + c["w"]/2.0
    cy = c["y"]
    above = [wd for wd in word_centers if wd["cy"] < cy + 10]
    if above:
        best = min(above, key=lambda wd: abs(wd["cy"] - cy) + abs(wd["cx"] - cx))
        c["label"] = best["text"]
    else:
        c["label"] = ""

# Load drawing font
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", FONT_SIZE)
except:
    font = ImageFont.load_default()

# Data mapping heuristics (edit values here if you want different content)
data = {
    "name": "John Doe",
    "dob": "07/06/1988",
    "gender": "Male",
    "address": "45 Greenway Ave",
    "city": "Bengaluru",
    "state": "KA",
    "zip": "560001",
    "phone": "9876543210",
    "email": "john@example.com",
    "preferred": "Phone",
    "emergency": "Alex Johnson",
    "relationship": "Spouse",
    "provider": "HDFC Ergo",
    "policy": "POL123456",
    "group": "GRP101",
    "policyholder": "Priya Kumar",
    "reason": "Lower back pain",
    "duration": "3 weeks",
    "diabetes": "Yes",
    "medications": "Metformin 500mg daily",
    "allergies": "Penicillin",
    "hospitalizations": "Appendectomy (2015)",
    "occupation": "Software Engineer",
    "date": datetime.datetime.now().strftime("%m/%d/%Y"),
    "signature": "John Doe"
}

# Helper: measure text width using textbbox
img_out = pil.copy()
draw = ImageDraw.Draw(img_out)
def text_width(text, font):
    bbox = draw.textbbox((0,0), text, font=font)
    return bbox[2] - bbox[0]

# Stamp values into detected boxes
stamped = 0
for c in filtered:
    lbl = (c.get("label") or "").lower()
    val = None
    # exact label match heuristics
    for k in data:
        if k in lbl or lbl in k:
            val = data[k]; break
    # fallback keywords
    if not val:
        if "name" in lbl: val = data.get("name")
        elif "dob" in lbl or "date" in lbl: val = data.get("dob") or data.get("date")
        elif "city" in lbl: val = data.get("city")
        elif "state" in lbl: val = data.get("state")
        elif "zip" in lbl or "postal" in lbl: val = data.get("zip")
        elif "phone" in lbl or "cell" in lbl: val = data.get("phone")
        elif "email" in lbl: val = data.get("email")
        elif "provider" in lbl or "insurance" in lbl: val = data.get("provider")
        elif "policy" in lbl: val = data.get("policy")
        elif "group" in lbl: val = data.get("group")
        elif "policyholder" in lbl: val = data.get("policyholder")
        elif "reason" in lbl: val = data.get("reason")
        elif "how long" in lbl or "how long have" in lbl: val = data.get("duration")
        elif "med" in lbl: val = data.get("medications")
        elif "allerg" in lbl: val = data.get("allergies")
        elif "hospital" in lbl: val = data.get("hospitalizations")
        elif "occupation" in lbl: val = data.get("occupation")
        elif "signature" in lbl or "sign" in lbl: val = data.get("signature")
    if c["type"] == "checkbox":
        # optionally draw check for 'yes' semantics (skip otherwise)
        if val and str(val).lower() in ("yes","y","true"):
            cx = c["x"] + c["w"]//2
            cy = c["y"] + c["h"]//2
            draw.line((cx-4, cy, cx, cy+6), fill=(0,0,0), width=2)
            draw.line((cx, cy+6, cx+8, cy-6), fill=(0,0,0), width=2)
            stamped += 1
        continue
    if not val:
        continue
    # wrap to box
    box_w = max(20, c["w"] - 8)
    words = str(val).split()
    lines = []
    line = ""
    for token in words:
        test = (line + " " + token).strip()
        if text_width(test, font) <= box_w:
            line = test
        else:
            if line:
                lines.append(line)
            line = token
    if line:
        lines.append(line)
    # draw up to MAX_LINES_PER_BOX
    x_px = c["x"] + 4
    y_px = c["y"] + 1
    line_h = int(FONT_SIZE * 1.15)
    for i,ln in enumerate(lines[:MAX_LINES_PER_BOX]):
        draw.text((x_px, y_px + i*line_h), ln, fill=(0,0,0), font=font)
    stamped += 1

print(f"Stamped {stamped} detected placeholders with values.")

# Save flattened PDF
out_path = "flattened_exact_filled.pdf"
buf = io.BytesIO()
img_out.save(buf, format="PDF")
buf.seek(0)
with open(out_path, "wb") as f:
    f.write(buf.read())

files.download(out_path)
print("Done — downloaded flattened_exact_filled.pdf")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Upload the Patient Intake template PDF (the blank form):


Saving Patient_Intake_form-Fillable.pdf to Patient_Intake_form-Fillable (22).pdf
Candidates detected: 1100 (merged heuristics)
Stamped 17 detected placeholders with values.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done — downloaded flattened_exact_filled.pdf
